
** Análisis de Datos I - 252
Unidad 2
Taller 2. Comparte tu Análisis univariado
POR ESTEBAN OLIVEROS **

---

Actividad: Análisis Univariado de una Variable en un Conjunto de Datos

Descripción:
En esta actividad, los estudiantes deberán seleccionar una columna (variable) de un conjunto de datos, describir su importancia, realizar un análisis univariado utilizando Python y presentar conclusiones basadas en los hallazgos.

Comparte el enlace a tu notebook  en tu repositorio de github (puede ser el enlace a tu github con acceso público o sólo a armandoordonez@gmail.com)

Instrucciones:
Descripción de la importancia de la columna (1 punto)

Explicar por qué la variable seleccionada es relevante dentro del conjunto de datos.
Indicar cómo podría influir en el análisis o en la toma de decisiones.
Análisis univariado en Python (2 puntos)

Generar estadísticas descriptivas (media, mediana, moda, desviación estándar, valores atípicos, etc.).
Visualizar la distribución de la variable usando histogramas, boxplots u otras gráficas adecuadas.
Identificar posibles sesgos o patrones dentro de los datos.
Conclusiones del análisis (2 puntos)

Resumir los hallazgos clave obtenidos en el análisis.
Mencionar implicaciones o próximos pasos que podrían derivarse de los resultados.


In [5]:
# ==========================================
# Bloque 1 — Preparación y metadatos del dataset SECOP Integrado
# Propósito: configurar entorno, definir palabras clave y municipios del Valle,
# y verificar las columnas reales del dataset rpmr-utcd vía metadatos.
# ==========================================

import os
import requests
import pandas as pd
import numpy as np
from typing import List, Optional

# --- Configuración base del dataset (SECOP Integrado) ---
DATASET_ID = "rpmr-utcd"
BASE_URL   = f"https://www.datos.gov.co/resource/{DATASET_ID}.json"
META_URL   = f"https://www.datos.gov.co/api/views/{DATASET_ID}.json"
APP_TOKEN  = os.getenv("SOCRATA_APP_TOKEN")  # opcional, mejora límites/estabilidad
HEADERS    = {"X-App-Token": APP_TOKEN} if APP_TOKEN else {}

# --- Palabras clave para identificar obras ---
KW_BASE = [
    "OBRA","OBRAS","CONSTRUCCI"
]
KW_EXTRA_15 = [
    "AMPLIAC","REASFALT","BACHEO",
]
# Unimos y quitamos duplicados manteniendo el orden
KEYWORDS_OBRAS = list(dict.fromkeys(KW_BASE + KW_EXTRA_15))

# --- Lista canónica de municipios del Valle del Cauca (incluye variantes con y sin tilde) ---
MUNICIPIOS_VALLE = [
    "SANTIAGO DE CALI","CALI","PALMIRA","YUMBO","JAMUNDI","JAMUNDÍ","CANDELARIA","PRADERA","FLORIDA",
    "VIJES","LA CUMBRE","DAGUA","RESTREPO","YOTOCO","CALIMA","EL DARIEN","EL DARÍEN","GUADALAJARA DE BUGA",
    "BUGA","BUGALAGRANDE","TULUA","TULUÁ","SEVILLA","CAICEDONIA","TRUJILLO","RIOFRIO","RIOFRÍO","SAN PEDRO",
    "GUACARI","GUACARÍ","GINEBRA","EL CERRITO","EL DOVIO","ROLDANILLO","LA UNION","LA UNIÓN","TORO","OBANDO",
    "ULLOA","VERSALLES","ZARZAL","CARTAGO","ANSERMANUEVO","ARGELIA","BOLIVAR","BOLÍVAR","EL CAIRO","EL AGUILA",
    "EL ÁGUILA","ANDALUCIA","ANDALUCÍA","BUENAVENTURA"
]

# --- Utilidad: lectura de metadatos/columnas reales del dataset ---
def get_columns_metadata(meta_url: str, headers: dict) -> pd.DataFrame:
    """Devuelve DataFrame con name, fieldName y type desde el API de metadatos."""
    r = requests.get(meta_url, headers=headers, timeout=60)
    r.raise_for_status()
    cols = r.json().get("columns", [])
    df_cols = pd.DataFrame(
        [(c.get("name"), c.get("fieldName"), c.get("dataTypeName")) for c in cols],
        columns=["name", "fieldName", "type"]
    ).dropna(subset=["fieldName"]).reset_index(drop=True)
    return df_cols

df_cols = get_columns_metadata(META_URL, HEADERS)
print("Columnas detectadas (fieldName → type):")
display(df_cols[["fieldName","type"]])

# --- Utilidad: detección robusta de nombres de campo por alias y búsqueda difusa ---
def pick_field(df_cols: pd.DataFrame,
               candidates: List[str],
               fallback_contains: List[str] = None,
               prefer_contains: List[str] = None) -> Optional[str]:
    """Elige el nombre real de columna en el dataset probando candidatos exactos y búsqueda difusa."""
    cols_l = df_cols["fieldName"].str.lower()
    columns_set = set(cols_l)

    # Intento exacto
    for c in candidates:
        if c and c.lower() in columns_set:
            return df_cols.loc[cols_l == c.lower(), "fieldName"].iloc[0]

    # Intento difuso por substrings
    fallback_contains = fallback_contains or []
    prefer_contains   = prefer_contains or []
    matches = []
    for f in cols_l:
        if all(tok in f for tok in fallback_contains):
            matches.append(f)
    if not matches:
        return None

    # Ranking por tokens preferidos y longitud
    def score(name: str):
        pref = sum(1 for tok in prefer_contains if tok in name)
        return (pref, -len(name))  # más preferencia y nombre más corto

    matches.sort(key=score, reverse=True)
    best_lower = matches[0]
    return df_cols.loc[cols_l == best_lower, "fieldName"].iloc[0]

# --- Campos que necesitaremos (se confirman aquí y se usan más adelante) ---
CAMPO_DEPTO = pick_field(df_cols, ["departamento"], fallback_contains=["depar"], prefer_contains=["depart"])
CAMPO_MPIO  = pick_field(df_cols, ["municipio"],     fallback_contains=["muni"],  prefer_contains=["municip"])
CAMPO_TIPO  = pick_field(df_cols, ["tipo_de_contrato","tipo_contrato","tipocontrato"], fallback_contains=["tipo","contrat"])
CAMPO_OBJ   = pick_field(df_cols, ["objeto_a_contratar","objeto_del_proceso","nombre_del_proceso","objeto"], fallback_contains=["obj"])
CAMPO_VALOR = pick_field(df_cols, ["valor_del_contrato","valor_total_adjudicacion","valor_total","valor"], fallback_contains=["valor","contrat"])
CAMPO_ENT   = pick_field(df_cols, ["nombre_de_la_entidad","entidad","entidad_contratante","nombre_entidad"], fallback_contains=["entid"])
CAMPO_PROV  = pick_field(df_cols, ["nombre_del_contratista","proveedor_adjudicado","proveedor"], fallback_contains=["prove","contrat"])
CAMPO_FI    = pick_field(df_cols, ["fecha_inicio_de_ejecucion","fecha_de_inicio","fecha_inicio","fecha_adjudicacion"], fallback_contains=["fecha","inici"])
CAMPO_FF    = pick_field(df_cols, ["fecha_fin_de_ejecucion","fecha_de_fin","fecha_fin","fecha_terminacion"], fallback_contains=["fecha","fin"])
CAMPO_ORD   = pick_field(df_cols, ["ordenentidad","nivel_entidad","nivel_ent"], fallback_contains=["orden","nivel"])
CAMPO_MOD   = pick_field(df_cols, ["modalidad_de_contratacion","modalidad"], fallback_contains=["modal"])

print("\nCampos seleccionados:")
print("Departamento:", CAMPO_DEPTO)
print("Municipio   :", CAMPO_MPIO)
print("Tipo        :", CAMPO_TIPO)
print("Objeto      :", CAMPO_OBJ)
print("Valor       :", CAMPO_VALOR)
print("Entidad     :", CAMPO_ENT)
print("Proveedor   :", CAMPO_PROV)
print("Fecha inicio:", CAMPO_FI)
print("Fecha fin   :", CAMPO_FF)
print("Orden ent.  :", CAMPO_ORD)
print("Modalidad   :", CAMPO_MOD)

# Aserciones mínimas para no seguir si faltan claves críticas
assert CAMPO_DEPTO or CAMPO_MPIO, "No se encontró ni departamento ni municipio."
assert CAMPO_OBJ or CAMPO_TIPO,  "No se encontró ni objeto ni tipo de contrato."
assert CAMPO_VALOR,              "No se encontró el campo de valor del contrato."


Columnas detectadas (fieldName → type):


,fieldName,type
0,nivel_entidad,text
1,codigo_entidad_en_secop,text
2,nombre_de_la_entidad,text
3,nit_de_la_entidad,text
4,departamento_entidad,text
5,municipio_entidad,text
6,estado_del_proceso,text
7,modalidad_de_contrataci_n,text
8,objeto_a_contratar,text
9,objeto_del_proceso,text



Campos seleccionados:
Departamento: departamento_entidad
Municipio   : municipio_entidad
Tipo        : tipo_de_contrato
Objeto      : objeto_a_contratar
Valor       : valor_contrato
Entidad     : nombre_de_la_entidad
Proveedor   : None
Fecha inicio: fecha_inicio_ejecuci_n
Fecha fin   : fecha_fin_ejecuci_n
Orden ent.  : nivel_entidad
Modalidad   : modalidad_de_contrataci_n


In [6]:
# ==========================================================
# Bloque 2 — Validación del enfoque: Valle + Obras (conteos + muestra)
# Propósito: construir WHERE para Valle + Obras (filtrando por tipo de contrato 'Obra') y
# EXCLUIR: prestacion de servicios / prestacion de servicios profesionales / interventoria.
# Validar conteos, traer 100 filas de muestra y revisar columnas/valores.
# ==========================================================

import requests
import time # Para reintentos básicos
import pandas as pd # Asegurar que pandas esté importado

# --- Utilidad: conteo server-side ---
def socrata_count(base_url: str, where: str = None, headers: dict = None) -> int:
    """Devuelve el número de registros que cumplen un WHERE en un dataset."""
    params = {"$select": "count(*)"}
    if where:
        params["$where"] = where
    headers = headers or {} # Usar headers proporcionados o un diccionario vacío

    try:
        # Usar HEADERS definidos en Bloque 1 (pasados como argumento)
        r = requests.get(base_url, params=params, headers=headers, timeout=60)
        r.raise_for_status()
        data = r.json()
        if data and isinstance(data, list) and data[0].get('count'):
            return int(data[0]['count'])
        # Si la respuesta no es la esperada pero no hubo error HTTP
        print(f"Advertencia: Respuesta de conteo inesperada: {data}")
        return 0
    except requests.exceptions.Timeout:
        print("Error al obtener conteo de Socrata: Tiempo de espera agotado.")
        return 0
    except requests.RequestException as e:
        print(f"Error al obtener conteo de Socrata: {e}")
        return 0 # Devolver 0 en caso de error


# --- Utilidad: GET con reintentos ---
def sodata_get(url: str, params: dict = None, headers: dict = None, timeout: int = 60, retries: int = 5):
    """Realiza una petición GET a Socrata con reintentos."""
    headers = headers or {}
    for i in range(retries):
        try:
            r = requests.get(url, params=params, headers=headers, timeout=timeout)
            r.raise_for_status()
            return r
        except requests.exceptions.Timeout:
            print(f"Intento {i+1}/{retries} fallido para {url}: Tiempo de espera agotado.")
            time.sleep(2 ** i) # Espera exponencial
        except requests.RequestException as e:
            print(f"Intento {i+1}/{retries} fallido para {url}: {e}")
            time.sleep(2 ** i) # Espera exponencial
    # Si todos los reintentos fallan, lanzar la última excepción
    print(f"Todos los {retries} reintentos fallaron para {url}.")
    r = requests.get(url, params=params, headers=headers, timeout=timeout) # Intenta una última vez para lanzar excepción
    r.raise_for_status()
    return r


# --- Construcción del filtro de obras (basado principalmente en tipo_de_contrato) ---
# Asegurarnos de que la cláusula de obra esté construida correctamente, solo basada en CAMPO_TIPO = 'Obra' si CAMPO_TIPO existe.
if CAMPO_TIPO:
    WHERE_OBRAS_INCLUSION = f"upper({CAMPO_TIPO}) = upper('Obra')"
elif CAMPO_OBJ: # Fallback si CAMPO_TIPO no existe
     like_obras = [f"upper({CAMPO_OBJ}) like upper('%{kw}%')" for kw in KEYWORDS_OBRAS]
     WHERE_OBRAS_INCLUSION = "(" + " OR ".join(like_obras) + ")"
else:
     raise ValueError("No se pudo construir cláusula de inclusión para 'obras'. Revisa CAMPO_TIPO/CAMPO_OBJ.")


# --- Filtro geográfico principal (departamento) con fallback por municipio ---
if CAMPO_DEPTO:
    # Incluir 'Valle del Cauca' y 'Valle' en la cláusula del departamento
    geo_clause = f"(upper({CAMPO_DEPTO}) = upper('Valle del Cauca') OR upper({CAMPO_DEPTO}) = upper('Valle'))"
elif CAMPO_MPIO: # Fallback por municipio si no hay campo departamento
    mpio_list = ", ".join("'" + m.replace("'", "''").upper() + "'" for m in MUNICIPIOS_VALLE)
    geo_clause = f"upper({CAMPO_MPIO}) IN ({mpio_list})"
else:
     raise ValueError("No se encontró ni campo de departamento ni campo de municipio para filtro geográfico.")


# --- Exclusión explícita de tipos/objetos no deseados ---
EXCLUDE_KEYWORDS = [
    "prestacion de servicios",
    "prestacion de servicios profesionales",
    "interventoria"
]
exclude_clauses = []
if CAMPO_TIPO:
    exclude_clauses += [f"upper({CAMPO_TIPO}) not like upper('%{kw}%')" for kw in EXCLUDE_KEYWORDS]
if CAMPO_OBJ:
    exclude_clauses += [f"upper({CAMPO_OBJ}) not like upper('%{kw}%')" for kw in EXCLUDE_KEYWORDS]

assert exclude_clauses, "No fue posible construir cláusulas de exclusión."

# --- WHERE final ---
# Combinamos el filtro geográfico, la inclusión de obras (tipo='Obra') y las exclusiones.
WHERE = f"{geo_clause} AND ({WHERE_OBRAS_INCLUSION})"
if exclude_clauses:
    WHERE += " AND (" + " AND ".join(exclude_clauses) + ")"


assert "None" not in WHERE # Validar que no queden "None" en el WHERE
print("WHERE SoQL:\n", WHERE)

# --- Conteos de control ---
# Pasar BASE_URL y HEADERS a la función socrata_count
total_dataset = socrata_count(BASE_URL, headers=HEADERS)
solo_valle    = socrata_count(BASE_URL, where=geo_clause, headers=HEADERS)
valle_obras   = socrata_count(BASE_URL, where=WHERE, headers=HEADERS)

print("\n===== MINI REPORTE DE COBERTURA =====")
print(f"Total dataset: {total_dataset:,}")
print(f"Solo geografía (Valle del Cauca o Valle): {solo_valle:,}")
print(f"Valle + Obras (Tipo='Obra') [excluyendo servicios/interventoría]: {valle_obras:,}")
print("=====================================\n")

# --- Muestra de validación (100 filas) ---
params_preview = {"$where": WHERE, "$limit": 100}

# Usar sodata_get con HEADERS
print("Usando sodata_get para la muestra.")
try:
    preview_response = sodata_get(BASE_URL, params=params_preview, headers=HEADERS, timeout=180)
    preview_response.raise_for_status() # Asegurarse de que haya respuesta exitosa
    df_preview = pd.DataFrame(preview_response.json())
except requests.RequestException as e:
    print(f"Error al obtener la muestra después de varios reintentos: {e}")
    df_preview = pd.DataFrame() # Inicializar vacío si falla

pd.set_option("display.max_columns", None)
print("Vista de 100 registros filtrados (para validar):")
display(df_preview.head(100))

if not df_preview.empty:
    print("\nNombres de columnas en la muestra:")
    print(sorted(df_preview.columns.tolist()))

    # --- Limpieza obligatoria mínima (en la muestra) ---
    # 1) convertir valor del contrato a numérico
    if CAMPO_VALOR in df_preview.columns:
        df_preview[CAMPO_VALOR] = pd.to_numeric(df_preview[CAMPO_VALOR], errors="coerce")

    # 2) eliminar filas con valor == 0 o NaN
    n_before = len(df_preview)
    df_preview = df_preview[df_preview[CAMPO_VALOR].fillna(0) > 0]

    # 3) eliminar contratista no definido (normalizado)
    def _norm(s):
        if not isinstance(s, str): return ""
        return (s.strip().lower()
                  .replace("á","a").replace("é","e").replace("í","i").replace("ó","o").replace("ú","u"))

    bad_contractor = {"no definido","no aplica","sin informacion","na","n/a","no registra","no reporta","no disponible"}
    if CAMPO_PROV in df_preview.columns:
        mask_bad = df_preview[CAMPO_PROV].map(_norm).isin(bad_contractor)
        df_preview = df_preview[~mask_bad]

    n_after = len(df_preview)
    print(f"\nLimpieza rápida de muestra → filas antes: {n_before} | después: {n_after}")

    # --- Chequeo rápido de municipios en la muestra (validación visual) ---
    if CAMPO_MPIO and CAMPO_MPIO in df_preview.columns:
        print("\nConteo por municipio en la muestra (top 20):")
        print(df_preview[CAMPO_MPIO].value_counts(dropna=False).head(20))
else:
    print("\nLa muestra está vacía o no se pudo descargar.")

WHERE SoQL:
 (upper(departamento_entidad) = upper('Valle del Cauca') OR upper(departamento_entidad) = upper('Valle')) AND (upper(tipo_de_contrato) = upper('Obra')) AND (upper(tipo_de_contrato) not like upper('%prestacion de servicios%') AND upper(tipo_de_contrato) not like upper('%prestacion de servicios profesionales%') AND upper(tipo_de_contrato) not like upper('%interventoria%') AND upper(objeto_a_contratar) not like upper('%prestacion de servicios%') AND upper(objeto_a_contratar) not like upper('%prestacion de servicios profesionales%') AND upper(objeto_a_contratar) not like upper('%interventoria%'))

===== MINI REPORTE DE COBERTURA =====
Total dataset: 19,766,430
Solo geografía (Valle del Cauca o Valle): 1,238,619
Valle + Obras (Tipo='Obra') [excluyendo servicios/interventoría]: 21,079

Usando sodata_get para la muestra.
Vista de 100 registros filtrados (para validar):


,nivel_entidad,codigo_entidad_en_secop,nombre_de_la_entidad,nit_de_la_entidad,departamento_entidad,municipio_entidad,estado_del_proceso,modalidad_de_contrataci_n,objeto_a_contratar,objeto_del_proceso,tipo_de_contrato,numero_del_contrato,numero_de_proceso,valor_contrato,nom_raz_social_contratista,url_contrato,origen,tipo_documento_proveedor,documento_proveedor,fecha_de_firma_del_contrato,fecha_inicio_ejecuci_n,fecha_fin_ejecuci_n
0,TERRITORIAL,276109011,VALLE DEL CAUCA - ALCALDÍA MUNICIPIO DE BUENAV...,890503483-2,Valle del Cauca,Buenaventura,Convocado,Selección Abreviada de Menor Cuantía (Ley 1150...,NO DEFINIDO,CONSTRUCCIoN DEL PAVIMENTO EN CONCRETO RiGIDO ...,Obra,24-11-14216005,SIV-SA-2024-0092,150000000,NO DEFINIDO,https://www.contratos.gov.co/consultas/detalle...,SECOPI,NO DEFINIDO,NO DEFINIDO,NaN,NaN,NaN
1,TERRITORIAL,276109011,VALLE DEL CAUCA - ALCALDÍA MUNICIPIO DE BUENAV...,890503483-2,Valle del Cauca,Buenaventura,Convocado,Selección Abreviada de Menor Cuantía (Ley 1150...,NO DEFINIDO,CONSTRUCCIoN PAVIMENTO EN\n CONCRETO RiGIDO EN...,Obra,24-11-14224352,SIV-SA-2024-0100,511245709,NO DEFINIDO,https://www.contratos.gov.co/consultas/detalle...,SECOPI,NO DEFINIDO,NO DEFINIDO,NaN,NaN,NaN
2,TERRITORIAL,276109011,VALLE DEL CAUCA - ALCALDÍA MUNICIPIO DE BUENAV...,890503483-2,Valle del Cauca,Buenaventura,Convocado,Selección Abreviada de Menor Cuantía (Ley 1150...,NO DEFINIDO,PREVENCIoN Y REDUCCIoN DEL RIESGO A TRAVeS DE ...,Obra,24-11-14225619,OPD-SA-2024-0101,220000000,NO DEFINIDO,https://www.contratos.gov.co/consultas/detalle...,SECOPI,NO DEFINIDO,NO DEFINIDO,NaN,NaN,NaN
3,TERRITORIAL,276616011,VALLE DEL CAUCA - ALCALDÍA MUNICIPIO DE RIOFRÍO,891900357,Valle del Cauca,Riofrío,Adjudicado,Selección Abreviada de Menor Cuantía (Ley 1150...,NO DEFINIDO,MEJORAMIENTO RECUPERACIoN Y MANTENIMIENTO DE L...,Obra,25-11-14461826,018-2025,262319398,NO DEFINIDO,https://www.contratos.gov.co/consultas/detalle...,SECOPI,NO DEFINIDO,NO DEFINIDO,NaN,NaN,NaN
4,TERRITORIAL,276845011,VALLE DEL CAUCA - ALCALDÍA MUNICIPIO DE ULLOA,899999281-2,Valle del Cauca,Ulloa,Convocado,Contratación Mínima Cuantía,NO DEFINIDO,MANTENIMIENTO Y EMBELLECIMIENTO DE EDIFICACIoN...,Obra,25-13-14457817,COP098-2025,13497637,NO DEFINIDO,https://www.contratos.gov.co/consultas/detalle...,SECOPI,NO DEFINIDO,NO DEFINIDO,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,TERRITORIAL,276000012,VALLE DEL CAUCA - UNIVERSIDAD DEL VALLE,890399010,Valle del Cauca,Cali,Celebrado,Régimen Especial,CUBIERTAS EN QUIMOTOP PARA MESONES DE LABORATORIO,CUBIERTAS EN QUIMOTOP PARA MESONES DE LABORATORIO,Obra,14-4-2884307,VRI-0725-14,14000522,CONTROL DE CONTAMINACION LTDA,https://www.contratos.gov.co/consultas/detalle...,SECOPI,Nit de Persona Jurídica,890326425,2014-07-14T00:00:00.000,2014-07-14T00:00:00.000,2014-11-11T00:00:00.000
96,TERRITORIAL,276000012,VALLE DEL CAUCA - UNIVERSIDAD DEL VALLE,890399010,Valle del Cauca,Cali,Celebrado,Régimen Especial,DISENO SITIO WEB CITCE EN JOOMIA Y DISENO CARP...,DISEÑO SITIO WEB CITCE EN JOOMIA Y DISEÑO CARP...,Obra,17-4-7369715,VRIN 36 I 134-2017,5400000,GUSTAVO ADOLFO GOMEZ SALAZAR,https://www.contratos.gov.co/consultas/detalle...,SECOPI,Cédula de Ciudadanía,14622603,2017-11-01T00:00:00.000,2017-11-01T00:00:00.000,2017-11-30T00:00:00.000
97,TERRITORIAL,276000012,VALLE DEL CAUCA - UNIVERSIDAD DEL VALLE,890399010,Valle del Cauca,Cali,Celebrado,Régimen Especial,MANTENIMIENTO PREVENTIVO Y CORRECTIVO DE LOS E...,MANTENIMIENTO PREVENTIVO Y CORRECTIVO DE LOS E...,Obra,13-4-2162140,SCRD-143-2013,3944000,HOME FITNESS LTDA,https://www.contratos.gov.co/consultas/detalle...,SECOPI,Nit de Persona Jurídica,900235801,2013-08-20T00:00:00.000,2013-08-20T00:00:00.000,2013-12-18T00:00:00.000
98,TERRITORIAL,276000012,VALLE DEL CAUCA - UNIVERSIDAD DEL VALLE,890399010,Valle del Cauca,Cali,Celebrado,Régimen Especial,SEGUNDA FASE DE INTERVENCION DEL EDIFICIO TULI...,SEGUNDA FASE DE INTERVENCIoN DEL EDIFICIO TULI...,Obra,13


Nombres de columnas en la muestra:
['codigo_entidad_en_secop', 'departamento_entidad', 'documento_proveedor', 'estado_del_proceso', 'fecha_de_firma_del_contrato', 'fecha_fin_ejecuci_n', 'fecha_inicio_ejecuci_n', 'modalidad_de_contrataci_n', 'municipio_entidad', 'nit_de_la_entidad', 'nivel_entidad', 'nom_raz_social_contratista', 'nombre_de_la_entidad', 'numero_de_proceso', 'numero_del_contrato', 'objeto_a_contratar', 'objeto_del_proceso', 'origen', 'tipo_de_contrato', 'tipo_documento_proveedor', 'url_contrato', 'valor_contrato']

Limpieza rápida de muestra → filas antes: 100 | después: 97

Conteo por municipio en la muestra (top 20):
municipio_entidad
Cali                   65
Roldanillo             14
Bugalagrande            4
Buenaventura            3
Tuluá                   3
Candelaria              2
Riofrío                 2
Ulloa                   1
Jamundí                 1
Guacarí                 1
Guadalajara de Buga     1
Name: count, dtype: int64


In [8]:
# ==============================================================
# Bloque 3
# Extracción paginada y preprocesamiento inicial
# ==============================================================
# Propósito: traer TODOS los registros filtrados (Valle + Obras),
# limpiar valores y contratistas, convertir fechas, guardar Parquet.

import math
from tqdm import tqdm
import gc
import unicodedata # Import unicodedata for accent removal

# --- Parámetros de paginación ---
PAGE_SIZE = 50000  # máximo recomendado por Socrata
total_rows = valle_obras
n_pages = math.ceil(total_rows / PAGE_SIZE)

print(f"Total esperado de registros Valle+Obras: {total_rows:,}")
print(f"Descargando en {n_pages} páginas de {PAGE_SIZE}...")

frames = []
for i in tqdm(range(n_pages), desc="Descargando páginas"):
    params = {
        "$where": WHERE,
        "$limit": PAGE_SIZE,
        "$offset": i * PAGE_SIZE
    }
    r = requests.get(BASE_URL, params=params, headers=HEADERS, timeout=600)
    r.raise_for_status()
    data = r.json()
    if not data:
        break
    frames.append(pd.DataFrame(data))

df_raw = pd.concat(frames, ignore_index=True)
print(f"DataFrame consolidado: {df_raw.shape}")

# --- Limpieza obligatoria ---
# 1) Valor numérico
df_raw[CAMPO_VALOR] = pd.to_numeric(df_raw[CAMPO_VALOR], errors="coerce")

# 2) Eliminar valor 0 o NaN
before_val = len(df_raw)
df_raw = df_raw[df_raw[CAMPO_VALOR].fillna(0) > 0]
after_val = len(df_raw)
print(f"Filas eliminadas por valor 0/NaN: {before_val - after_val}")

# 3) Eliminar contratistas no definidos
bad_contractor = {"no definido","no aplica","sin informacion","na","n/a","no registra","no reporta"}
def _norm(s):
    if not isinstance(s, str): return ""
    return (s.strip().lower()
              .replace("á","a").replace("é","e").replace("í","i").replace("ó","o").replace("ú","u"))

if CAMPO_PROV in df_raw.columns:
    mask_bad = df_raw[CAMPO_PROV].map(_norm).isin(bad_contractor)
    before_contr = len(df_raw)
    df_raw = df_raw[~mask_bad]
    after_contr = len(df_raw)
    print(f"Filas eliminadas por contratista inválido: {before_contr - after_contr}")

# 4) Convertir fechas
for campo, nombre in [(CAMPO_FI,"inicio"), (CAMPO_FF,"fin")]:
    if campo and campo in df_raw.columns:
        df_raw[campo] = pd.to_datetime(df_raw[campo], errors="coerce")

# --- Normalizar municipios (quitar tildes y convertir a mayúsculas) ---
def normalize_municipio(text):
    if not isinstance(text, str):
        return text
    # Remove accents
    text = ''.join(c for c in unicodedata.normalize('NFD', text) if unicodedata.category(c) != 'Mn')
    # Convert to uppercase
    return text.upper()

if CAMPO_MPIO and CAMPO_MPIO in df_raw.columns:
    df_raw[CAMPO_MPIO] = df_raw[CAMPO_MPIO].apply(normalize_municipio)


# --- Guardado en Parquet (Drive) ---
OUTPUT_PATH = "/content/drive/MyDrive/secop_valle_obras.parquet"
df_raw.to_parquet(OUTPUT_PATH, index=False)
print(f"Guardado en: {OUTPUT_PATH}")

# Liberar memoria auxiliar
del frames
gc.collect()

# --- Validación municipios ---
if CAMPO_MPIO in df_raw.columns:
    print("\nCobertura de municipios del Valle (conteo de obras):")
    conteo_mpios = df_raw[CAMPO_MPIO].value_counts() # No need for .str.upper() here as it's already normalized
    display(conteo_mpios.head(50))

    # Normalize MUNICIPIOS_VALLE for comparison
    normalized_municipios_valle = [normalize_municipio(m) for m in MUNICIPIOS_VALLE]

    missing = [m for m in normalized_municipios_valle if m not in conteo_mpios.index]
    print(f"\nMunicipios del Valle SIN registros detectados (normalized): {missing}")

#.-.-.-.-.    HASTA AQUÍ TODO FUNCIONA  .-.-.-.-.-.-.
#.-.-.-.-.    VIERNES                   .-.-.-.-..-.-
#.-.-.-.-.    29 DE AGOSTO DE 2025      .-.-.-.-.-.-.
#.-.-.-.-.    12:28 PM                  .-.-.-.-.-.-.
#.-.-.-.-.    Valle+Obras: 21,079       .-.-.-.-.-.-.

Total esperado de registros Valle+Obras: 21,079
Descargando en 1 páginas de 50000...


Descargando páginas: 100%|██████████| 1/1 [00:02<00:00,  2.84s/it]


DataFrame consolidado: (21079, 22)
Filas eliminadas por valor 0/NaN: 36
Guardado en: /content/drive/MyDrive/secop_valle_obras.parquet

Cobertura de municipios del Valle (conteo de obras):


,count
municipio_entidad,
CALI,6657
BUENAVENTURA,1419
TULUA,868
PALMIRA,826
ZARZAL,669
YUMBO,575
CANDELARIA,551
FLORIDA,512
EL CERRITO,492



Municipios del Valle SIN registros detectados (normalized): ['SANTIAGO DE CALI', 'EL DARIEN', 'EL DARIEN', 'BUGA']


In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [10]:
import os
import pandas as pd

file_path = '/content/drive/MyDrive/secop_valle_obras.parquet'
if os.path.exists(file_path):
    df_raw = pd.read_parquet(file_path)
    print(f"DataFrame loaded from '{file_path}' with shape: {df_raw.shape}")
else:
    print(f"Error: File not found at {file_path}")
    # Since the file is not found, the subtask cannot be completed.
    # We will finish the task with a failure status.
    raise FileNotFoundError(f"File not found: {file_path}")

DataFrame loaded from '/content/drive/MyDrive/secop_valle_obras.parquet' with shape: (21043, 22)


In [ ]:
import pandas as pd

# Function to format currency
def format_cop(value):
    if pd.isna(value):
        return value
    # Format as Colombian Pesos: $, separating thousands with '.', and millions with "'"
    # Handle large numbers by casting to integer first to avoid scientific notation issues
    try:
        value = int(value)
        return f"${value:,.0f}".replace(",", "'").replace(".", ",")
    except (ValueError, TypeError):
        return value # Return original value if formatting fails

# Apply formatting for display purposes in head()
# Create a copy for display to avoid modifying the original DataFrame column type
df_display = df_raw.copy()
if CAMPO_VALOR in df_display.columns:
    df_display[CAMPO_VALOR] = df_display[CAMPO_VALOR].apply(format_cop)

display(df_display.head(100))
display(df_raw.info()) # Display info on the original DataFrame to show actual dtypes
display(df_raw.describe()) # Display describe on the original DataFrame for numerical summary

#.-.-.-.-.    HASTA AQUÍ TODO FUNCIONA  .-.-.-.-.-.-.
#.-.-.-.-.    VIERNES                   .-.-.-.-..-.-
#.-.-.-.-.    29 DE AGOSTO DE 2025      .-.-.-.-.-.-.
#.-.-.-.-.    12:57 PM                  .-.-.-.-.-.-.


,nivel_entidad,codigo_entidad_en_secop,nombre_de_la_entidad,nit_de_la_entidad,departamento_entidad,municipio_entidad,estado_del_proceso,modalidad_de_contrataci_n,objeto_a_contratar,objeto_del_proceso,tipo_de_contrato,numero_del_contrato,numero_de_proceso,valor_contrato,nom_raz_social_contratista,url_contrato,origen,tipo_documento_proveedor,documento_proveedor,fecha_de_firma_del_contrato,fecha_inicio_ejecuci_n,fecha_fin_ejecuci_n
0,TERRITORIAL,276109011,VALLE DEL CAUCA - ALCALDÍA MUNICIPIO DE BUENAV...,890503483-2,Valle del Cauca,BUENAVENTURA,Convocado,Selección Abreviada de Menor Cuantía (Ley 1150...,NO DEFINIDO,CONSTRUCCIoN DEL PAVIMENTO EN CONCRETO RiGIDO ...,Obra,24-11-14216005,SIV-SA-2024-0092,$150'000'000,NO DEFINIDO,https://www.contratos.gov.co/consultas/detalle...,SECOPI,NO DEFINIDO,NO DEFINIDO,None,NaT,NaT
1,TERRITORIAL,276109011,VALLE DEL CAUCA - ALCALDÍA MUNICIPIO DE BUENAV...,890503483-2,Valle del Cauca,BUENAVENTURA,Convocado,Selección Abreviada de Menor Cuantía (Ley 1150...,NO DEFINIDO,CONSTRUCCIoN PAVIMENTO EN\n CONCRETO RiGIDO EN...,Obra,24-11-14224352,SIV-SA-2024-0100,$511'245'709,NO DEFINIDO,https://www.contratos.gov.co/consultas/detalle...,SECOPI,NO DEFINIDO,NO DEFINIDO,None,NaT,NaT
2,TERRITORIAL,276109011,VALLE DEL CAUCA - ALCALDÍA MUNICIPIO DE BUENAV...,890503483-2,Valle del Cauca,BUENAVENTURA,Convocado,Selección Abreviada de Menor Cuantía (Ley 1150...,NO DEFINIDO,PREVENCIoN Y REDUCCIoN DEL RIESGO A TRAVeS DE ...,Obra,24-11-14225619,OPD-SA-2024-0101,$220'000'000,NO DEFINIDO,https://www.contratos.gov.co/consultas/detalle...,SECOPI,NO DEFINIDO,NO DEFINIDO,None,NaT,NaT
3,TERRITORIAL,276616011,VALLE DEL CAUCA - ALCALDÍA MUNICIPIO DE RIOFRÍO,891900357,Valle del Cauca,RIOFRIO,Adjudicado,Selección Abreviada de Menor Cuantía (Ley 1150...,NO DEFINIDO,MEJORAMIENTO RECUPERACIoN Y MANTENIMIENTO DE L...,Obra,25-11-14461826,018-2025,$262'319'398,NO DEFINIDO,https://www.contratos.gov.co/consultas/detalle...,SECOPI,NO DEFINIDO,NO DEFINIDO,None,NaT,NaT
4,TERRITORIAL,276845011,VALLE DEL CAUCA - ALCALDÍA MUNICIPIO DE ULLOA,899999281-2,Valle del Cauca,ULLOA,Convocado,Contratación Mínima Cuantía,NO DEFINIDO,MANTENIMIENTO Y EMBELLECIMIENTO DE EDIFICACIoN...,Obra,25-13-14457817,COP098-2025,$13'497'637,NO DEFINIDO,https://www.contratos.gov.co/consultas/detalle...,SECOPI,NO DEFINIDO,NO DEFINIDO,None,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,TERRITORIAL,276000012,VALLE DEL CAUCA - UNIVERSIDAD DEL VALLE,890399010,Valle del Cauca,CALI,Celebrado,Régimen Especial,SEGUNDA FASE DE INTERVENCION DEL EDIFICIO TULI...,SEGUNDA FASE DE INTERVENCIoN DEL EDIFICIO TULI...,Obra,13-4-1989524,SOLICITUD DE OFERTAS 42-2012,$282'719'551,ALVARO LAZARO DEL VALLE,https://www.contratos.gov.co/consultas/detalle...,SECOPI,Nit de Persona Natural,10545570,2012-12-04T00:00:00.000,2013-06-24,2013-11-24
96,TERRITORIAL,276000012,VALLE DEL CAUCA - UNIVERSIDAD DEL VALLE,890399010,Valle del Cauca,CALI,Celebrado,Régimen Especial,SERVICIOS PROFESIONALES PARA REALIZAR ACTIVIDA...,SERVICIOS PROFESIONALES PARA REALIZAR ACTIVIDA...,Obra,20-4-10554865,MIC-VRIN-06-0046-2020,$7'000'000,JOSE MANUEL ROSERO GIRALDO,https://www.contratos.gov.co/consultas/detalle...,SECOPI,Nit de Persona Natural,1151956166,2020-03-02T00:00:00.000,2020-03-05,2020-05-31
97,TERRITORIAL,276000012,VALLE DEL CAUCA - UNIVERSIDAD DEL VALLE,890399010,Valle del Cauca,CALI,Descartado,Licitación Pública,NO DEFINIDO,¿REMODELACIoN DE LAS INSTALACIONES DE LAS BATE...,Obra,15-1-141363,LP_003_20154,$772'296'808,NO DEFINIDO,https://www.contratos.gov.co/consultas/detalle...,SECOPI,NO DEFINIDO,NO DEFINIDO,None,NaT,NaT
98,TERRITORIAL,276000012,VALLE DEL CAUCA - UNIVERSIDAD DEL VALLE,890399010,Valle del Cauca,CALI,Descartado,Licitación Pública,NO DEFINIDO,REMODELACIoN Y DOTACIoN DEL LABORATORIO DE ALT...,Obra,14-1-129623,Licitación Pública 045 - 20143,$298'569'576,NO DEFINIDO,https://www.contratos.gov.co/consultas/detalle...,SECOPI,NO DEFINIDO,NO DEFINIDO,None,

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21043 entries, 0 to 21042
Data columns (total 22 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   nivel_entidad                21043 non-null  object        
 1   codigo_entidad_en_secop      21043 non-null  object        
 2   nombre_de_la_entidad         21043 non-null  object        
 3   nit_de_la_entidad            21043 non-null  object        
 4   departamento_entidad         21043 non-null  object        
 5   municipio_entidad            21043 non-null  object        
 6   estado_del_proceso           21043 non-null  object        
 7   modalidad_de_contrataci_n    21043 non-null  object        
 8   objeto_a_contratar           21043 non-null  object        
 9   objeto_del_proceso           21043 non-null  object        
 10  tipo_de_contrato             21043 non-null  object        
 11  numero_del_contrato          21043 non-nu

None

,valor_contrato,fecha_inicio_ejecuci_n,fecha_fin_ejecuci_n
count,2.104300e+04,15851,16123
mean,1.401092e+09,2017-07-06 08:04:23.453410048,2017-12-02 14:09:27.586677504
min,1.000000e+00,2004-07-08 00:00:00,2004-08-07 00:00:00
25%,1.735654e+07,2014-05-19 12:00:00,2014-09-11 00:00:00
50%,6.000000e+07,2017-10-10 00:00:00,2018-01-06 00:00:00
75%,3.199978e+08,2020-11-27 00:00:00,2021-05-19 12:00:00
max,3.480000e+12,2025-08-26 00:00:00,2038-08-04 00:00:00
std,2.593907e+10,NaN,NaN


In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np # Import numpy directly
import os

# Assuming df_raw is already loaded from the parquet file
# file_path = '/content/drive/MyDrive/secop_valle_obras.parquet'
# if 'df_raw' not in globals():
#     if os.path.exists(file_path):
#         df_raw = pd.read_parquet(file_path)
#         print(f"DataFrame loaded with shape: {df_raw.shape}")
#     else:
#         print(f"Error: File not found at {file_path}")
#         df_raw = pd.DataFrame() # Create an empty DataFrame to avoid further errors

if not df_raw.empty:
    # Ensure CAMPO_VALOR is available (assuming it was defined in a previous cell)
    if 'CAMPO_VALOR' not in globals() or CAMPO_VALOR is None or CAMPO_VALOR not in df_raw.columns:
         # Re-define utility functions and variables from Block 1 if not already defined
        def get_columns_metadata(meta_url, headers):
            r = requests.get(meta_url, headers=headers, timeout=60)
            r.raise_for_status()
            cols = r.json().get("columns", [])
            df_cols = pd.DataFrame(
                [(c.get("name"), c.get("fieldName"), c.get("dataTypeName")) for c in cols],
                columns=["name", "fieldName", "type"]
            ).dropna(subset=["fieldName"]).reset_index(drop=True)
            return df_cols

        def pick_field(df_cols, candidates, fallback_contains=None, prefer_contains=None):
            cols_l = df_cols["fieldName"].str.lower()
            columns_set = set(cols_l)
            for c in candidates:
                if c and c.lower() in columns_set:
                    return df_cols.loc[cols_l == c.lower(), "fieldName"].iloc[0]
            fallback_contains = fallback_contains or []
            prefer_contains   = prefer_contains or []
            matches = []
            for f in cols_l:
                if all(tok in f for tok in fallback_contains):
                    matches.append(f)
            if not matches:
                return None
            def score(name):
                pref = sum(1 for tok in prefer_contains if tok in name)
                return (pref, -len(name))
            matches.sort(key=score, reverse=True)
            best_lower = matches[0]
            return df_cols.loc[cols_l == best_lower, "fieldName"].iloc[0]

        DATASET_ID = "rpmr-utcd"
        META_URL   = f"https://www.datos.gov.co/api/views/{DATASET_ID}.json"
        HEADERS    = {}
        df_cols = get_columns_metadata(META_URL, HEADERS)
        CAMPO_VALOR = pick_field(df_cols, ["valor_del_contrato","valor_total_adjudicacion","valor_total","valor"], fallback_contains=["valor","contrat"])

    # --- Data Categorization ---
    # Define thresholds in COP
    threshold_millions = 1_000_000
    threshold_billions = 1_000_000_000
    threshold_hundred_billions = 100_000_000_000

    def categorize_value(value):
        if pd.isna(value):
            return 'Desconocido'
        if value <= 1 * threshold_billions: # Up to 1,000 million (1 billion)
            return 'Bajo (0 - 1.000 millones)'
        elif value <= 10 * threshold_billions: # 1,000 million to 10,000 million (1 to 10 billion)
            return 'Medio (1.000 - 10.000 millones)'
        elif value <= 100 * threshold_billions: # 10,000 million to 100,000 million (10 to 100 billion)
            return 'Alto (10.000 - 100.000 millones)'
        else: # Above 100,000 million (100 billion)
            return 'Muy Alto (> 100.000 millones)'

    df_raw['valor_categoria'] = df_raw[CAMPO_VALOR].apply(categorize_value)

    # --- Currency Formatting for Plot Axes ---
    def format_cop_plotly(value):
        if value is None or pd.isna(value):
            return ''
        value = int(value) # Ensure integer for formatting
        if value >= 1_000_000_000_000:
            return f"${value / 1_000_000_000_000:.0f} Billón"
        elif value >= 1_000_000_000:
            billions = value // 1_000_000_000
            millions = (value % 1_000_000_000) // 1_000_000
            if millions > 0:
                 return f"${billions}.{millions:03d}'000.000" # Format example: $1.000'000.000
            else:
                 return f"${billions}.000'000.000"

        elif value >= 1_000_000:
             thousands = (value % 1_000_000) // 1000
             millions = value // 1_000_000
             if thousands > 0:
                 return f"${millions}.{thousands:03d}.000" # Format example: $1.000.000
             else:
                 return f"${millions}.000.000"

        elif value >= 1000:
            return f"${value:,.0f}".replace(",", ".") # Format example: $1.000

        else:
            return f"${value:,.0f}"

    # --- Generate Histograms for each Value Category ---
    category_order = ['Bajo (0 - 1.000 millones)', 'Medio (1.000 - 10.000 millones)', 'Alto (10.000 - 100.000 millones)', 'Muy Alto (> 100.000 millones)']

    for category in category_order:
        df_category = df_raw[df_raw['valor_categoria'] == category].copy()

        if not df_category.empty:
            fig_hist_cat = px.histogram(df_category, x=CAMPO_VALOR, nbins=100, log_y=True,
                                        title=f'<b>Distribution of Contract Values: {category}</b>',
                                        labels={CAMPO_VALOR: 'Valor del Contrato', 'count': 'Frecuencia'})

            # Apply custom currency formatting to x-axis ticks
            # Adjust tick values based on the range of the current category
            min_val = df_category[CAMPO_VALOR].min()
            max_val = df_category[CAMPO_VALOR].max()
            # Avoid log10(0) or negative values
            if min_val > 0:
                tick_vals = [10**i for i in range(int(np.log10(min_val)), int(np.log10(max_val)) + 1)]
                tick_text = [format_cop_plotly(val) for val in tick_vals]
            else:
                # Handle cases where min_val is 0 or very small
                tick_vals = [0] + [10**i for i in range(int(np.log10(1)), int(np.log10(max_val)) + 1)] # Start from 1 or a small number
                tick_text = [format_cop_plotly(val) for val in tick_vals]


            fig_hist_cat.update_layout(
                xaxis = dict(
                    tickmode = 'array',
                    tickvals = tick_vals,
                    ticktext = tick_text
                )
            )
            fig_hist_cat.show()
        else:
            print(f"No data found for category: {category}. Skipping histogram.")

else:
    print("DataFrame df_raw is empty. Skipping visualizations.")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Define the list of categorical column names to analyze
categorical_cols = []
if 'CAMPO_TIPO' in globals() and CAMPO_TIPO:
    categorical_cols.append(CAMPO_TIPO)
if 'CAMPO_MOD' in globals() and CAMPO_MOD:
    categorical_cols.append(CAMPO_MOD)
if 'CAMPO_MPIO' in globals() and CAMPO_MPIO:
    categorical_cols.append(CAMPO_MPIO)
if 'CAMPO_ORD' in globals() and CAMPO_ORD:
    categorical_cols.append(CAMPO_ORD)
if 'estado_del_proceso' in df_raw.columns: # Check if the column exists in the dataframe
    categorical_cols.append('estado_del_proceso')

print("Categorical columns selected for univariate analysis:", categorical_cols)

Categorical columns selected for univariate analysis: ['tipo_de_contrato', 'modalidad_de_contrataci_n', 'municipio_entidad', 'nivel_entidad', 'estado_del_proceso']


In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

# Assuming df_raw and CAMPO_VALOR are already defined and df_raw has 'valor_categoria'
if not df_raw.empty:
    # Ensure CAMPO_VALOR and 'valor_categoria' are available
    if 'CAMPO_VALOR' not in globals() or CAMPO_VALOR is None or CAMPO_VALOR not in df_raw.columns or 'valor_categoria' not in df_raw.columns:
         print("Required columns (CAMPO_VALOR or valor_categoria) not available. Cannot generate box plots.")
    else:
        # Define currency formatting function (assuming it's not globally available)
        def format_cop_plotly(value):
            if value is None or pd.isna(value):
                return ''
            value = int(value) # Ensure integer for formatting
            if value >= 1_000_000_000_000:
                return f"${value / 1_000_000_000_000:.0f} Billón"
            elif value >= 1_000_000_000:
                billions = value // 1_000_000_000
                millions = (value % 1_000_000_000) // 1_000_000
                if millions > 0:
                     return f"${billions}.{millions:03d}'000.000" # Format example: $1.000'000.000
                else:
                     return f"${billions}.000'000.000"

            elif value >= 1_000_000:
                 thousands = (value % 1_000_000) // 1000
                 millions = value // 1_000_000
                 if thousands > 0:
                     return f"${millions}.{thousands:03d}.000" # Format example: $1.000.000
                 else:
                     return f"${millions}.000.000"

            elif value >= 1000:
                return f"${value:,.0f}".replace(",", ".") # Format example: $1.000

            else:
                return f"${value:,.0f}"

        # 1. Box plot for all data
        fig_box_all = px.box(df_raw, y=CAMPO_VALOR,
                             title='<b>Box Plot of All Contract Values</b>',
                             labels={CAMPO_VALOR: 'Valor del Contrato'})

        # Apply custom currency formatting to y-axis ticks
        fig_box_all.update_layout(
             yaxis = dict(
                type='log', # Use log scale due to skewness
                tickmode = 'array',
                tickvals = [10**i for i in range(int(np.log10(df_raw[CAMPO_VALOR].min() + 1)), int(np.log10(df_raw[CAMPO_VALOR].max())) + 1)], # Adjust tick values based on data range
                ticktext = [format_cop_plotly(10**i) for i in range(int(np.log10(df_raw[CAMPO_VALOR].min() + 1)), int(np.log10(df_raw[CAMPO_VALOR].max())) + 1)]
             )
        )
        fig_box_all.show()

        # 2. Box plots for each value category
        category_order = ['Bajo (0 - 1.000 millones)', 'Medio (1.000 - 10.000 millones)', 'Alto (10.000 - 100.000 millones)', 'Muy Alto (> 100.000 millones)']

        for category in category_order:
            df_category = df_raw[df_raw['valor_categoria'] == category].copy()

            if not df_category.empty:
                fig_box_cat = px.box(df_category, y=CAMPO_VALOR,
                                     title=f'<b>Box Plot of Contract Values: {category}</b>',
                                     labels={CAMPO_VALOR: 'Valor del Contrato'})

                # Apply custom currency formatting to y-axis ticks
                # Adjust tick values based on the range of the current category
                min_val = df_category[CAMPO_VALOR].min()
                max_val = df_category[CAMPO_VALOR].max()
                # Avoid log10(0) or negative values, and ensure there are ticks
                tick_vals = []
                tick_text = []
                if min_val > 0:
                    tick_vals = [10**i for i in range(int(np.log10(min_val)), int(np.log10(max_val)) + 1)]
                    tick_text = [format_cop_plotly(val) for val in tick_vals]
                elif max_val > 0: # Handle cases where min_val is 0 or very small but max_val is positive
                     tick_vals = [0] + [10**i for i in range(int(np.log10(1)), int(np.log10(max_val)) + 1)] # Start from 1 or a small number
                     tick_text = [format_cop_plotly(val) for val in tick_vals]


                fig_box_cat.update_layout(
                     yaxis = dict(
                        type='log', # Use log scale
                        tickmode = 'array',
                        tickvals = tick_vals,
                        ticktext = tick_text
                     )
                )
                fig_box_cat.show()
            else:
                print(f"No data found for category: {category}. Skipping box plot.")

else:
    print("DataFrame df_raw is empty. Skipping box plots.")

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Assuming df_raw and categorical_cols are already defined

if not df_raw.empty and 'categorical_cols' in globals() and categorical_cols:
    for col in categorical_cols:
        if col in df_raw.columns:
            # Calculate value counts
            counts = df_raw[col].value_counts().reset_index()
            counts.columns = [col, 'count']

            # Generate bar plot using Plotly Express
            fig = px.bar(counts, x='count', y=col, orientation='h',
                         title=f'<b>Distribution of {col}</b>',
                         labels={'count': 'Count', col: col},
                         text='count') # Add text labels on bars

            fig.update_layout(yaxis={'categoryorder':'total ascending'}) # Order bars by count
            fig.show()
        else:
            print(f"Column '{col}' not found in DataFrame. Skipping plot.")
else:
    print("DataFrame df_raw is empty or categorical_cols is not defined/empty. Skipping plots.")

In [4]:
# Check for duplicate values in 'numero_del_contrato' in df_raw (secop_valle_obras)
if 'df_raw' in globals() and not df_raw.empty:
    if 'numero_del_contrato' in df_raw.columns:
        duplicates = df_raw[df_raw.duplicated(subset=['numero_del_contrato'], keep=False)]
        print(f"Number of duplicate 'numero_del_contrato' values in df_raw: {len(duplicates)}")

        # Count unique values in 'numero_del_contrato'
        unique_count = df_raw['numero_del_contrato'].nunique()
        print(f"Number of unique 'numero_del_contrato' values in df_raw: {unique_count}")
    else:
        print("Column 'numero_del_contrato' not found in df_raw.")
else:
    print("DataFrame df_raw is not loaded or is empty.")

DataFrame df_raw is not loaded or is empty.


In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import os
import requests

# Re-define utility functions and variables from Block 1 if not already defined
# This ensures CAMPO_* variables are available
def get_columns_metadata(meta_url: str, headers: dict) -> pd.DataFrame:
    """Devuelve DataFrame con name, fieldName y type desde el API de metadatos."""
    r = requests.get(meta_url, headers=headers, timeout=60)
    r.raise_for_status()
    cols = r.json().get("columns", [])
    df_cols = pd.DataFrame(
        [(c.get("name"), c.get("fieldName"), c.get("dataTypeName")) for c in cols],
        columns=["name", "fieldName", "type"]
    ).dropna(subset=["fieldName"]).reset_index(drop=True)
    return df_cols

def pick_field(df_cols: pd.DataFrame,
               candidates: list[str],
               fallback_contains: list[str] = None,
               prefer_contains: list[str] = None) -> str | None:
    """Elige el nombre real de columna en el dataset probando candidatos exactos y búsqueda difusa."""
    cols_l = df_cols["fieldName"].str.lower()
    columns_set = set(cols_l)

    # Intento exacto
    for c in candidates:
        if c and c.lower() in columns_set:
            return df_cols.loc[cols_l == c.lower(), "fieldName"].iloc[0]

    # Intento difuso por substrings
    fallback_contains = fallback_contains or []
    prefer_contains   = prefer_contains or []
    matches = []
    for f in cols_l:
        if all(tok in f for tok in fallback_contains):
            matches.append(f)
    if not matches:
        return None

    # Ranking por tokens preferidos y longitud
    def score(name: str):
        pref = sum(1 for tok in prefer_contains if tok in name)
        return (pref, -len(name))  # más preferencia y nombre más corto

    matches.sort(key=score, reverse=True)
    best_lower = matches[0]
    return df_cols.loc[cols_l == best_lower, "fieldName"].iloc[0]

# Re-define these variables if they are not in the current environment
DATASET_ID = "rpmr-utcd"
META_URL   = f"https://www.datos.gov.co/api/views/{DATASET_ID}.json"
HEADERS    = {} # Assuming HEADERS are not critical for this step if APP_TOKEN is not set

df_cols = get_columns_metadata(META_URL, HEADERS)

CAMPO_DEPTO = pick_field(df_cols, ["departamento"], fallback_contains=["depar"], prefer_contains=["depart"])
CAMPO_MPIO  = pick_field(df_cols, ["municipio"],     fallback_contains=["muni"],  prefer_contains=["municip"])
CAMPO_TIPO  = pick_field(df_cols, ["tipo_de_contrato","tipo_contrato","tipocontrato"], fallback_contains=["tipo","contrat"])
CAMPO_OBJ   = pick_field(df_cols, ["objeto_a_contratar","objeto_del_proceso","nombre_del_proceso","objeto"], fallback_contains=["obj"])
CAMPO_VALOR = pick_field(df_cols, ["valor_del_contrato","valor_total_adjudicacion","valor_total","valor"], fallback_contains=["valor","contrat"])
CAMPO_ENT   = pick_field(df_cols, ["nombre_de_la_entidad","entidad","entidad_contratante","nombre_entidad"], fallback_contains=["entid"])
CAMPO_PROV  = pick_field(df_cols, ["nombre_del_contratista","proveedor_adjudicado","proveedor"], fallback_contains=["prove","contrat"])
CAMPO_FI    = pick_field(df_cols, ["fecha_inicio_de_ejecucion","fecha_de_inicio","fecha_inicio","fecha_adjudicacion"], fallback_contains=["fecha","inici"])
CAMPO_FF    = pick_field(df_cols, ["fecha_fin_de_ejecucion","fecha_de_fin","fecha_fin","fecha_terminacion"], fallback_contains=["fecha","fin"])
CAMPO_ORD   = pick_field(df_cols, ["ordenentidad","nivel_entidad","nivel_ent"], fallback_contains=["orden","nivel"])
CAMPO_MOD   = pick_field(df_cols, ["modalidad_de_contratacion","modalidad"], fallback_contains=["modal"])


# Define currency formatting function for Plotly axes
def format_cop_plotly(value):
    if value is None or pd.isna(value):
        return ''
    value = int(value) # Ensure integer for formatting
    if value >= 1_000_000_000_000:
        return f"${value / 1_000_000_000_000:.0f} Billón"
    elif value >= 1_000_000_000:
        billions = value // 1_000_000_000
        millions = (value % 1_000_000_000) // 1_000_000
        if millions > 0:
             return f"${billions}.{millions:03d}'000.000" # Format example: $1.000'000.000
        else:
             return f"${billions}.000'000.000"

    elif value >= 1_000_000:
         thousands = (value % 1_000_000) // 1000
         millions = value // 1_000_000
         if thousands > 0:
             return f"${millions}.{thousands:03d}.000" # Format example: $1.000.000
         else:
             return f"${millions}.000.000"

    elif value >= 1000:
        return f"${value:,.0f}".replace(",", ".") # Format example: $1.000

    else:
        return f"${value:,.0f}"

# Load the dataframe if not already loaded
if 'df_raw' not in globals():
    file_path = '/content/drive/MyDrive/secop_valle_obras.parquet'
    if os.path.exists(file_path):
        df_raw = pd.read_parquet(file_path)
        print(f"DataFrame loaded with shape: {df_raw.shape}")
    else:
        print(f"Error: File not found at {file_path}")
        df_raw = pd.DataFrame() # Create an empty DataFrame to avoid further errors


if not df_raw.empty:
    # Ensure required columns are available
    required_cols = [CAMPO_VALOR, CAMPO_FI, CAMPO_TIPO, CAMPO_MOD, CAMPO_MPIO, CAMPO_ENT]
    for col_var in required_cols:
        if col_var is None or col_var not in df_raw.columns:
            print(f"Warning: Required column '{col_var}' not available in DataFrame. Skipping related plots.")


    # 1. Scatter plot of valor_contrato vs. fecha_inicio_ejecuci_n (Option 1)
    if CAMPO_VALOR and CAMPO_FI and CAMPO_VALOR in df_raw.columns and CAMPO_FI in df_raw.columns:
        # Convert date column to datetime if not already
        if not pd.api.types.is_datetime64_any_dtype(df_raw[CAMPO_FI]):
            df_raw[CAMPO_FI] = pd.to_datetime(df_raw[CAMPO_FI], errors='coerce')

        # Remove rows with NaT in date or NaN/0 in value for plotting
        plot_df_scatter = df_raw.dropna(subset=[CAMPO_FI, CAMPO_VALOR]).copy()
        plot_df_scatter = plot_df_scatter[plot_df_scatter[CAMPO_VALOR].fillna(0) > 0]

        if not plot_df_scatter.empty:
            fig_scatter = px.scatter(plot_df_scatter, x=CAMPO_FI, y=CAMPO_VALOR,
                                     title=f'<b>Contract Value vs. Start Date</b>',
                                     labels={CAMPO_FI: 'Start Date', CAMPO_VALOR: 'Valor del Contrato'})

            # Apply log scale and currency formatting to y-axis
            fig_scatter.update_layout(
                 yaxis = dict(
                    type='log', # Set y-axis to log scale
                    tickmode = 'array',
                    tickvals = [10**i for i in range(int(np.log10(plot_df_scatter[CAMPO_VALOR].min() + 1)), int(np.log10(plot_df_scatter[CAMPO_VALOR].max())) + 1)], # Adjust tick values based on data range
                    ticktext = [format_cop_plotly(10**i) for i in range(int(np.log10(plot_df_scatter[CAMPO_VALOR].min() + 1)), int(np.log10(plot_df_scatter[CAMPO_VALOR].max())) + 1)]
                 )
            )
            fig_scatter.show()
        else:
            print(f"Skipping scatter plot: No valid data points after cleaning for {CAMPO_VALOR} and {CAMPO_FI}.")
    else:
        print(f"Skipping scatter plot: Required columns for scatter plot not available or found.")

    print("-" * 50) # Separator

    # 2. Box plot of valor_contrato by municipio_entidad (Top N) (Option 4)
    if CAMPO_MPIO and CAMPO_VALOR and CAMPO_MPIO in df_raw.columns and CAMPO_VALOR in df_raw.columns:
        # Consider top N municipalities
        N = 20
        # Calculate total value per municipality for ranking
        total_value_by_mpio = df_raw.groupby(CAMPO_MPIO)[CAMPO_VALOR].sum().nlargest(N)
        top_mpios = total_value_by_mpio.index.tolist() # Get municipalities with top total value

        df_top_mpios = df_raw[df_raw[CAMPO_MPIO].isin(top_mpios)].copy()

        if not df_top_mpios.empty:
            fig_box_mpio = px.box(df_top_mpios, y=CAMPO_MPIO, x=CAMPO_VALOR,
                                  category_orders={CAMPO_MPIO: top_mpios},
                                  title=f'<b>{CAMPO_VALOR} by Top {N} Municipalities (by Total Value)</b>',
                                  labels={CAMPO_VALOR: 'Valor del Contrato', CAMPO_MPIO: CAMPO_MPIO})

            # Apply log scale and currency formatting to x-axis
            fig_box_mpio.update_layout(
                 xaxis = dict(
                    type='log', # Set x-axis to log scale
                    tickmode = 'array',
                    tickvals = [10**i for i in range(int(np.log10(df_top_mpios[CAMPO_VALOR].min() + 1)), int(np.log10(df_top_mpios[CAMPO_VALOR].max())) + 1)], # Adjust tick values based on data range
                    ticktext = [format_cop_plotly(10**i) for i in range(int(np.log10(df_top_mpios[CAMPO_VALOR].min() + 1)), int(np.log10(df_top_mpios[CAMPO_VALOR].max())) + 1)]
                 )
            )
            fig_box_mpio.show()
        else:
            print(f"Skipping municipality box plot: No data for top {N} municipalities after filtering.")
    else:
        print(f"Skipping municipality box plot: Required columns '{CAMPO_VALOR}' or '{CAMPO_MPIO}' not available or found.")

    print("-" * 50) # Separator


    # 3. Box plot of valor_contrato by modalidad_de_contrataci_n (Option 3)
    if CAMPO_VALOR and CAMPO_MOD and CAMPO_VALOR in df_raw.columns and CAMPO_MOD in df_raw.columns:
        fig_box_mod = px.box(df_raw, y=CAMPO_MOD, x=CAMPO_VALOR,
                             title=f'<b>{CAMPO_VALOR} by {CAMPO_MOD}</b>',
                             labels={CAMPO_VALOR: 'Valor del Contrato', CAMPO_MOD: CAMPO_MOD})

        # Apply log scale and currency formatting to x-axis
        fig_box_mod.update_layout(
             xaxis = dict(
                type='log', # Set x-axis to log scale
                tickmode = 'array',
                tickvals = [10**i for i in range(int(np.log10(df_raw[CAMPO_VALOR].min() + 1)), int(np.log10(df_raw[CAMPO_VALOR].max())) + 1)], # Adjust tick values based on data range
                ticktext = [format_cop_plotly(10**i) for i in range(int(np.log10(df_raw[CAMPO_VALOR].min() + 1)), int(np.log10(df_raw[CAMPO_VALOR].max())) + 1)]
             )
        )
        fig_box_mod.show()
    else:
         print(f"Skipping box plot for {CAMPO_MOD}: Required columns '{CAMPO_VALOR}' or '{CAMPO_MOD}' not available or found.")

    print("-" * 50) # Separator


    # 4. Box plot of valor_contrato by nombre_de_la_entidad (Top N) (Option 9)
    if CAMPO_ENT and CAMPO_VALOR and CAMPO_ENT in df_raw.columns and CAMPO_VALOR in df_raw.columns:
        # Consider top N entities
        N = 20
        # Calculate total value per entity for ranking
        total_value_by_ent = df_raw.groupby(CAMPO_ENT)[CAMPO_VALOR].sum().nlargest(N)
        top_ents = total_value_by_ent.index.tolist() # Get entities with top total value

        df_top_ents = df_raw[df_raw[CAMPO_ENT].isin(top_ents)].copy()

        if not df_top_ents.empty:
            fig_box_ent = px.box(df_top_ents, y=CAMPO_ENT, x=CAMPO_VALOR,
                                  category_orders={CAMPO_ENT: top_ents},
                                  title=f'<b>{CAMPO_VALOR} by Top {N} Entities (by Total Value)</b>',
                                  labels={CAMPO_VALOR: 'Valor del Contrato', CAMPO_ENT: CAMPO_ENT})

            # Apply log scale and currency formatting to x-axis
            fig_box_ent.update_layout(
                 xaxis = dict(
                    type='log', # Set x-axis to log scale
                    tickmode = 'array',
                    tickvals = [10**i for i in range(int(np.log10(df_top_ents[CAMPO_VALOR].min() + 1)), int(np.log10(df_top_ents[CAMPO_VALOR].max())) + 1)], # Adjust tick values based on data range
                    ticktext = [format_cop_plotly(10**i) for i in range(int(np.log10(df_top_ents[CAMPO_VALOR].min() + 1)), int(np.log10(df_top_ents[CAMPO_VALOR].max())) + 1)]
                 )
            )
            fig_box_ent.show()
        else:
             print(f"Skipping entity box plot: No data for top {N} entities after filtering.")
    else:
        print(f"Skipping entity box plot: Required columns '{CAMPO_VALOR}' or '{CAMPO_ENT}' not available or found.")

    print("-" * 50) # Separator


    # 5. Average Contract Value Over Time (Option 10)
    if CAMPO_VALOR and CAMPO_FI and CAMPO_VALOR in df_raw.columns and CAMPO_FI in df_raw.columns:
        # Convert date column to datetime if not already
        if not pd.api.types.is_datetime64_any_dtype(df_raw[CAMPO_FI]):
            df_raw[CAMPO_FI] = pd.to_datetime(df_raw[CAMPO_FI], errors='coerce')

        # Remove rows with NaT in date or NaN/0 in value for plotting
        plot_df_ts = df_raw.dropna(subset=[CAMPO_FI, CAMPO_VALOR]).copy()
        plot_df_ts = plot_df_ts[plot_df_ts[CAMPO_VALOR].fillna(0) > 0]

        if not plot_df_ts.empty:
            # Resample by year and calculate the mean value
            avg_value_by_year = plot_df_ts.set_index(CAMPO_FI)[CAMPO_VALOR].resample('YS').mean().reset_index()
            avg_value_by_year.columns = [CAMPO_FI, 'Average_Valor_Contrato']

            fig_line_avg = px.line(avg_value_by_year, x=CAMPO_FI, y='Average_Valor_Contrato',
                                  title='<b>Average Contract Value Over Time (Yearly)</b>',
                                  labels={CAMPO_FI: 'Year', 'Average_Valor_Contrato': 'Average Valor del Contrato'})

            # Apply currency formatting to y-axis
            fig_line_avg.update_layout(
                 yaxis = dict(
                    tickmode = 'array',
                    # Generate ticks based on the data range in the resampled data
                    tickvals = [i for i in np.linspace(avg_value_by_year['Average_Valor_Contrato'].min(), avg_value_by_year['Average_Valor_Contrato'].max(), 10)], # Example: 10 ticks evenly spaced
                    ticktext = [format_cop_plotly(i) for i in np.linspace(avg_value_by_year['Average_Valor_Contrato'].min(), avg_value_by_year['Average_Valor_Contrato'].max(), 10)]
                 )
            )

            fig_line_avg.show()

            # Optional: Resample by month and calculate the mean value (can be noisy)
            # avg_value_by_month = plot_df_ts.set_index(CAMPO_FI)[CAMPO_VALOR].resample('MS').mean().reset_index()
            # avg_value_by_month.columns = [CAMPO_FI, 'Average_Valor_Contrato']

            # fig_line_avg_month = px.line(avg_value_by_month, x=CAMPO_FI, y='Average_Valor_Contrato',
            #                       title='<b>Average Contract Value Over Time (Monthly)</b>',
            #                       labels={CAMPO_FI: 'Month', 'Average_Valor_Contrato': 'Average Valor del Contrato'})
            # fig_line_avg_month.show()

        else:
            print(f"Skipping average value over time plot: No valid data points after cleaning for {CAMPO_VALOR} and {CAMPO_FI}.")
    else:
        print(f"Skipping average value over time plot: Required columns '{CAMPO_VALOR}' or '{CAMPO_FI}' not available or found.")


else:
    print("DataFrame df_raw is empty. Skipping bivariate analysis.")

--------------------------------------------------


--------------------------------------------------


--------------------------------------------------


--------------------------------------------------


## Calculate contract duration

### Subtask:
Calculate the difference between the 'fecha_fin_ejecuci_n' and 'fecha_inicio_ejecuci_n' columns, convert the duration to months, and store it in a new column named 'meses_ejecucion'. Handle any missing or invalid dates appropriately.


In [ ]:
import numpy as np

# Ensure date columns are in datetime format (already done in Block 3, but re-checked for safety)
if CAMPO_FI in df_raw.columns and not pd.api.types.is_datetime64_any_dtype(df_raw[CAMPO_FI]):
    df_raw[CAMPO_FI] = pd.to_datetime(df_raw[CAMPO_FI], errors='coerce')
if CAMPO_FF in df_raw.columns and not pd.api.types.is_datetime64_any_dtype(df_raw[CAMPO_FF]):
    df_raw[CAMPO_FF] = pd.to_datetime(df_raw[CAMPO_FF], errors='coerce')

# Calculate the duration in days
# Use .dt.days to get the difference in days. This will result in NaT for rows with NaT in either date column.
if CAMPO_FI in df_raw.columns and CAMPO_FF in df_raw.columns:
    duration_days = (df_raw[CAMPO_FF] - df_raw[CAMPO_FI]).dt.days
    # Convert duration from days to months (approximate)
    # Handle potential division by zero or NaT results from duration_days
    df_raw['meses_ejecucion'] = duration_days / 30.44
else:
    print("Warning: Required date columns for duration calculation are not available.")
    df_raw['meses_ejecucion'] = np.nan # Add a column with NaN if date columns are missing

# Display the first few rows with the new column to verify
display(df_raw[['fecha_inicio_ejecuci_n', 'fecha_fin_ejecuci_n', 'meses_ejecucion']].head())


,fecha_inicio_ejecuci_n,fecha_fin_ejecuci_n,meses_ejecucion
0,NaT,NaT,NaN
1,NaT,NaT,NaN
2,NaT,NaT,NaN
3,NaT,NaT,NaN
4,NaT,NaT,NaN


**Reasoning**:
Generate a scatter plot to visualize the relationship between 'valor_contrato' and 'meses_ejecucion', applying log scale and custom currency formatting to the y-axis, as requested in the subtask.



In [ ]:
import plotly.express as px
import numpy as np
import pandas as pd # Ensure pandas is imported

# Assuming df_raw and CAMPO_VALOR are already defined and df_raw has 'meses_ejecucion'
if not df_raw.empty and 'meses_ejecucion' in df_raw.columns and CAMPO_VALOR in df_raw.columns:
    # Remove rows with NaN in meses_ejecucion or NaN/0 in CAMPO_VALOR for plotting
    plot_df_scatter_duration = df_raw.dropna(subset=['meses_ejecucion', CAMPO_VALOR]).copy()
    plot_df_scatter_duration = plot_df_scatter_duration[plot_df_scatter_duration[CAMPO_VALOR].fillna(0) > 0]

    if not plot_df_scatter_duration.empty:
        # Define duration ranges and categorize the data
        def categorize_duration(months):
            if pd.isna(months):
                return 'Desconocido'
            elif months <= 2: # New range: 0-2 months
                return '0-2 months'
            elif months <= 12: # Adjusted range: 2-12 months
                return '2-12 months'
            elif months <= 24: # Existing range: 12-24 months
                return '12-24 months'
            else: # Existing range: 24+ months
                return '24+ months'

        plot_df_scatter_duration['duration_category'] = plot_df_scatter_duration['meses_ejecucion'].apply(categorize_duration)

        # Define the order of duration categories for plotting
        duration_category_order = ['0-2 months', '2-12 months', '12-24 months', '24+ months']

        # Generate a scatter plot for each duration category
        for category in duration_category_order:
            df_category_duration = plot_df_scatter_duration[plot_df_scatter_duration['duration_category'] == category].copy()

            if not df_category_duration.empty:
                fig_scatter_duration = px.scatter(df_category_duration, x='meses_ejecucion', y=CAMPO_VALOR,
                                                  title=f'<b>Contract Value vs. Duration: {category}</b>',
                                                  labels={'meses_ejecucion': 'Duration (Months)', CAMPO_VALOR: 'Contract Value'})

                # Apply log scale and custom currency formatting to y-axis
                min_val = df_category_duration[CAMPO_VALOR].min()
                max_val = df_category_duration[CAMPO_VALOR].max()

                # Avoid log10(0) or negative values for tick calculation
                tick_vals = []
                tick_text = []
                if min_val > 0:
                    tick_vals = [10**i for i in range(int(np.log10(min_val)), int(np.log10(max_val)) + 1)]
                    tick_text = [format_cop_plotly(val) for val in tick_vals]
                elif max_val > 0: # Handle cases where min_val is 0 or very small but max_val is positive
                     tick_vals = [0] + [10**i for i in range(int(np.log10(1)), int(np.log10(max_val)) + 1)] # Start from 1 or a small number
                     tick_text = [format_cop_plotly(val) for val in tick_vals]

                fig_scatter_duration.update_layout(
                     yaxis = dict(
                        type='log', # Set y-axis to log scale
                        tickmode = 'array',
                        tickvals = tick_vals,
                        ticktext = tick_text
                     )
                )
                fig_scatter_duration.show()
            else:
                print(f"Skipping scatter plot: No data found for duration category: {category}.")

    else:
        print("Skipping scatter plot: No valid data points after cleaning for 'meses_ejecucion' and CAMPO_VALOR.")
else:
    print("Skipping scatter plot: DataFrame df_raw is empty or required columns ('meses_ejecucion', CAMPO_VALOR) are not available.")

print("-" * 50) # Separator

--------------------------------------------------


In [ ]:
# 1. Define Sectors and Keywords
sectors_and_keywords = {
    'educación': [
        'institución educativa', 'escolar', 'educativa', 'colegio', 'academia', 'estudiantes',
        'enseñanza', 'aprendizaje', 'capacitación', 'formación', 'universidad',
        'escuela', 'pedagógico', 'docente', 'aulas', 'campus',
        # Added misspellings/typos and variations:
        'institucion', 'educasion', 'colegioo', 'estudiante', 'ensenanza', 'apendizaje'
    ],
    'salud': [
        'puesto de salud', 'salud', 'emergencia', 'hospital', 'clínica', 'quirófano', 'centro de salud', 'lesiones','trauma','pandemia', 'covid',
        'médico', 'enfermería', 'paciente', 'tratamiento', 'prevención',
        'sanitario', 'asistencial', 'consultorio', 'farmacia', 'ambulancia',
        # Added misspellings/typos and variations:
        'clinica', 'medicco', 'enfermeria', 'paciente', 'tratamiento', 'prevencion'
    ],
    'deporte': [
        'cancha', 'estadio', 'coliseo', 'gradería', 'piscina',
        'deportivo', 'entrenamiento', 'competición', 'atlético', 'gimnasio',
        'polideportivo', 'pista', 'ejercicio', 'recreación', 'liga',
        # Added misspellings/typos and variations:
        'cancha', 'estadioo', 'coliseo', 'graderia', 'piscinaa', 'deportivvo', 'entrenamiento', 'competicion', 'atletico', 'gimnasio'
    ],
    'vivienda': [
        'vivienda', 'parque', 'casa', 'apartamento', 'conjunto habitacional',
        'residencial', 'hogar', 'inmobiliario', 'urbanización', 'habitacional', 'lote', 'solar',
        # Added misspellings/typos and variations:
        'viviendaa', 'parquee', 'casasa', 'apartamentoo', 'habitacional', 'residencial', 'urbanizacion',
    ],
    'cultura': [
        'teatro', 'biblioteca', 'cine',
        'cultural', 'arte', 'museo', 'historia', 'patrimonio',
        'artístico', 'escenario', 'exposición', 'galería', 'evento', 'festival', 'cultura'
        # Added misspellings/typos and variations:
        'teatroo', 'bibliotecaa', 'cinee', 'cultural', 'artte', 'musseo', 'historiaa', 'patrimonioo', 'artistico', 'exposicion'
    ],
    'seguridad': [
        'video', 'vigilancia', 'blindado', 'policia', 'ejercito', 'armada', 'fuerza aérea', 'paz', 'conflicto',
        'seguridad', 'protección', 'defensa', 'orden público', 'criminalidad',
        'patrullaje', 'alarma', 'cámaras', 'control', 'prevención',
        # Added misspellings/typos and variations:
        'videoo', 'vigilanciaa', 'blindadoo', 'policiaa', 'ejercitoo', 'armadaa', 'fuerza aerea', 'pazz', 'conflictoo', 'seguridaad', 'proteccion'
    ],
    'transporte': [
        'via', 'vías', 'puente', 'carretria', 'doblecalada', 'bacheo', 'asfalto', 'derrumbe', 'viaducto',
        'transporte', 'movilidad', 'carretera', 'autopista', 'calzada',
        'semaforización', 'señalización', 'peatonal', 'vehicular', 'logística',
        # Added misspellings/typos and variations:
        'viaa', 'vias', 'puentee', 'carreteria', 'doblecalzada', 'bacheoo', 'asfaltoo', 'derrumbee', 'viaductoo', 'transporte', 'movilidad'
    ],
    'servicios públicos': [
        'acueducto', 'alcantarillado', 'energía eléctrica', 'planta de tratamiento', 'ptar', 'ptab', 'alumbrado', 'agua potable', 'teléfonos',
        'servicio público', 'saneamiento', 'electrificación', 'residuos', 'basuras',
        'gas', 'telecomunicaciones', 'redes', 'infraestructura', 'domiciliario',
        # Added misspellings/typos and variations:
        'acueductoo', 'alcantarilladoo', 'energia electrica', 'planta de tratamiento', 'ptarr', 'ptabb', 'alumbrao', 'agua potablee', 'telefonoss', 'servicio publico'
    ],
    'tecnología': [
        'telecomunicaciones', 'internet', 'computadores', 'celulares', 'video', 'impresoras', 'red', 'fotovoltaico', 'laboratorio', 'ciencia', 'tecnología', 'innovación',
        'digital', 'software', 'hardware', 'sistemas', 'informática',
        'desarrollo', 'datos', 'conectividad', 'virtual', 'equipo',
        # Added misspellings/typos and variations:
        'telecomunicaciones', 'internett', 'computadoress', 'celularess', 'videoo', 'impresorass', 'redd', 'fotovoltaicoo', 'laboratorioo', 'cienciaa', 'tecnologia', 'innovacionn'
    ],
    'mejoramiento': [
        'adecuaciones', 'mantenimiento', # Corrected typo from manteniemiento
        'reparación', 'restauración', 'remodelación', 'rehabilitación', 'renovación',
        'ajustes', 'arreglo', 'optimización', 'modernización', 'actualización',
        # Added misspellings/typos and variations:
        'adecuacioness', 'mantenimientoo', 'reparacion', 'restauracion', 'remodelacion', 'rehabilitacion', 'renovacion', 'ajustess', 'arregloo', 'optimizacion', 'modernizacion'
    ],
    'medio ambiente': [
        'rio', 'lago', 'laguna', 'humedal', 'bosque', 'montaña', 'arboles', 'flora', 'fauna', 'silvestre',
        'ambiental', 'ecológico', 'naturaleza', 'conservación', 'biodiversidad',
        'contaminación', 'residuos', 'reciclaje', 'sostenibilidad', 'climático',
        # Added misspellings/typos and variations:
        'rioo', 'lagoo', 'lagunaa', 'humedall', 'bosquee', 'montanaa', 'arboless', 'floraa', 'faunaa', 'silvestree', 'ambientall', 'ecologico', 'naturalezaa', 'conservacion', 'biodiversidad'
    ]
}

print("Sectors and keywords defined.")

# Note: Accent removal will be handled in the categorization logic (next step)

Sectors and keywords defined.


In [ ]:
# 2. Categorize Contracts into Sectors
# Assuming df_raw and CAMPO_OBJ are defined and sectors_and_keywords is defined

# Ensure CAMPO_OBJ is available and handle missing values
if CAMPO_OBJ is None or CAMPO_OBJ not in df_raw.columns:
    print(f"Error: 'Objeto' column ('{CAMPO_OBJ}') not found in DataFrame. Skipping categorization.")
else:
    df_raw['objeto_processed'] = df_raw[CAMPO_OBJ].fillna('').astype(str).str.lower()

    # Function to find the first matching sector for a given text
    def find_sector(text, sectors_and_keywords):
        for sector, keywords in sectors_and_keywords.items():
            if any(keyword.lower() in text for keyword in keywords):
                return sector
        return 'otro' # Assign to 'otro' if no keywords are found

    # Apply the function to categorize each contract
    df_raw['sector'] = df_raw['objeto_processed'].apply(lambda x: find_sector(x, sectors_and_keywords))

    print("Contracts categorized into sectors.")
    display(df_raw['sector'].value_counts())

    # Drop the temporary processed column
    df_raw = df_raw.drop(columns=['objeto_processed'], errors='ignore')

Contracts categorized into sectors.


,count
sector,
otro,6318
educación,2637
transporte,2305
vivienda,1666
servicios públicos,1637
salud,1404
mejoramiento,1251
deporte,1185
medio ambiente,812


In [ ]:
import nltk
from nltk.corpus import stopwords
from collections import Counter
import string
import pandas as pd

# Assuming df_raw and CAMPO_OBJ are defined and df_raw has a 'sector' column

# Filter the DataFrame for contracts in the 'otro' sector
df_otro_sector = df_raw[df_raw['sector'] == 'otro'].copy()

if not df_otro_sector.empty:
    # Ensure CAMPO_OBJ is available and handle missing values in the filtered data
    if CAMPO_OBJ is None or CAMPO_OBJ not in df_otro_sector.columns:
        print(f"Error: 'Objeto' column ('{CAMPO_OBJ}') not found in the filtered DataFrame.")
    else:
        df_otro_sector[CAMPO_OBJ] = df_otro_sector[CAMPO_OBJ].fillna('').astype(str)

        # Tokenize and clean text (reusing the function if defined, or defining it here)
        def tokenize_and_clean(text):
            # Convert to lowercase
            text = text.lower()
            # Remove punctuation
            text = text.translate(str.maketrans('', '', string.punctuation))
            # Tokenize
            tokens = nltk.word_tokenize(text)
            return tokens

        # Process text for the 'otro' sector
        all_tokens_otro = df_otro_sector[CAMPO_OBJ].apply(tokenize_and_clean).sum()

        # Remove stop words (reusing stop_words set if defined, or defining it here)
        # Assuming stop_words from a previous cell (tj03wl0H1tWy) is available
        # If not, define it: stop_words = set(stopwords.words('spanish') + stopwords.words('english'))
        if 'stop_words' not in globals():
             nltk.download('stopwords', quiet=True)
             stop_words = set(stopwords.words('spanish') + stopwords.words('english'))


        filtered_tokens_otro = [word for word in all_tokens_otro if word not in stop_words and len(word) > 1] # Remove single characters

        # Calculate word frequencies
        word_counts_otro = Counter(filtered_tokens_otro)

        # Display top 100 most frequent words
        N = 100
        print(f"\nTop {N} most frequent words in '{CAMPO_OBJ}' for the 'otro' sector:")
        for word, count in word_counts_otro.most_common(N):
            print(f"{word}: {count}")
else:
    print("No contracts found in the 'otro' sector.")


Top 100 most frequent words in 'objeto_a_contratar' for the 'otro' sector:
definido: 4973
municipio: 480
valle: 363
cauca: 294
adecuacion: 198
mejoramiento: 115
sede: 94
realizar: 91
municipal: 90
corregimiento: 87
calle: 85
carrera: 75
instalacion: 70
limpieza: 67
obras: 64
zona: 57
quebrada: 57
centro: 55
servicios: 54
pavimento: 54
apoyo: 53
san: 53
obra: 51
departamento: 51
suministro: 49
cauce: 47
capacidad: 47
reposicion: 46
sector: 45
zarzal: 43
cali: 42
descolmatacion: 41
piso: 40
instalaciones: 40
contratar: 39
proyecto: 39
edificio: 39
secretaria: 38
urbana: 38
contrato: 38
buenaventura: 37
prestar: 36
costo: 33
cerrito: 33
santiago: 33
calles: 33
adecuación: 32
atencion: 31
alcaldia: 31
pedro: 30
carreras: 30
comuna: 29
senalizacion: 29
distrito: 29
tramos: 28
florida: 28
sevilla: 28
gestion: 27
reparcheo: 27
enlucimiento: 27
puntos: 27
servicio: 26
identificados: 26
buga: 26
central: 25
social: 25
concreto: 24
area: 24
10: 24
horizontal: 24
oficina: 23
11: 23
recursos: 23


In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import os

# Assuming df_raw, CAMPO_VALOR, CAMPO_FI, and 'sector' are already defined

if not df_raw.empty and CAMPO_VALOR in df_raw.columns and CAMPO_FI in df_raw.columns and 'sector' in df_raw.columns:
    # --- Remove outlier contract with value of 3 billion ---
    outlier_value = 3_000_000_000 # Define the outlier value
    initial_rows = len(df_raw)
    df_raw = df_raw[df_raw[CAMPO_VALOR] != outlier_value].copy()
    rows_after_removal = len(df_raw)
    print(f"Removed {initial_rows - rows_after_removal} contract(s) with value exactly {outlier_value}.")
    # --------------------------------------------------------


    # Ensure date column is in datetime format
    if not pd.api.types.is_datetime64_any_dtype(df_raw[CAMPO_FI]):
        df_raw[CAMPO_FI] = pd.to_datetime(df_raw[CAMPO_FI], errors='coerce')

    # Remove rows with NaT in date or NaN/0 in value for aggregation and plotting
    plot_df_ts_sector = df_raw.dropna(subset=[CAMPO_FI, CAMPO_VALOR]).copy()
    plot_df_ts_sector = plot_df_ts_sector[plot_df_ts_sector[CAMPO_VALOR].fillna(0) > 0]

    if not plot_df_ts_sector.empty:
        # Aggregate total value by sector and month using alternative method
        plot_df_ts_sector['Year'] = plot_df_ts_sector[CAMPO_FI].dt.year
        plot_df_ts_sector['Month'] = plot_df_ts_sector[CAMPO_FI].dt.month

        sector_value_by_month = plot_df_ts_sector.groupby(['sector', 'Year', 'Month'])[CAMPO_VALOR].sum().reset_index()
        sector_value_by_month.columns = ['sector', 'Year', 'Month', 'Total_Valor_Contrato'] # Column is correctly renamed here
        sector_value_by_month['Date'] = pd.to_datetime(sector_value_by_month[['Year', 'Month']].assign(day=1))

        # Sort by sector and date for correct cumulative sum calculation and plotting
        sector_value_by_month = sector_value_by_month.sort_values(by=['sector', 'Date'])

        # Calculate cumulative sum for each sector - Use the renamed column 'Total_Valor_Contrato'
        sector_value_by_month['Cumulative_Valor_Contrato'] = sector_value_by_month.groupby('sector')['Total_Valor_Contrato'].cumsum()

        # Define currency formatting function for Plotly axes (redefine if not globally available)
        def format_cop_plotly(value):
            if value is None or pd.isna(value):
                return ''
            value = int(value) # Ensure integer for formatting
            if value >= 1000000000000:
                return f"${value / 1000000000000:.0f} Billón"
            elif value >= 1000000000:
                billions = value // 1000000000
                millions = (value % 1000000000) // 1000000
                if millions > 0:
                     return f"${billions}.{millions:03d}'000.000" # Format example: $1.000'000.000
                else:
                     return f"${billions}.000'000.000"

            elif value >= 1000000:
                 thousands = (value % 1000000) // 1000
                 millions = value // 1000000
                 if thousands > 0:
                     return f"${millions}.{thousands:03d}.000" # Format example: $1.000.000
                 else:
                     return f"${millions}.000.000"

            elif value >= 1000:
                return f"${value:,.0f}".replace(",", ".") # Format example: $1.000

            else:
                return f"${value:,.0f}"

        # --- Plots ---

        # Calculate total cumulative value per sector for legend ordering
        cumulative_total_by_sector = sector_value_by_month.groupby('sector')['Cumulative_Valor_Contrato'].max().sort_values(ascending=False)
        sector_order = cumulative_total_by_sector.index.tolist()

        # 1. Cumulative Total Contract Value by Sector Over Time (Monthly)
        fig_line_sector_cumulative = px.line(sector_value_by_month, x='Date', y='Cumulative_Valor_Contrato', color='sector',
                                            category_orders={'sector': sector_order}, # Order legend by cumulative total
                                            title='<b>Valor Acumulado Total del Contrato por Sector a lo largo del Tiempo (Mensual) - OBRAS EN EL VALLE</b>',
                                            labels={'Date': 'Fecha', 'Cumulative_Valor_Contrato': 'Valor Acumulado Total del Contrato', 'sector': 'Sector'})

        # Apply currency formatting to y-axis
        fig_line_sector_cumulative.update_layout(
             yaxis = dict(
                tickmode = 'array',
                # Generate ticks based on the data range
                tickvals = [i for i in np.linspace(sector_value_by_month['Cumulative_Valor_Contrato'].min(), sector_value_by_month['Cumulative_Valor_Contrato'].max(), 10)], # Example: 10 ticks evenly spaced
                ticktext = [format_cop_plotly(i) for i in np.linspace(sector_value_by_month['Cumulative_Valor_Contrato'].min(), sector_value_by_month['Cumulative_Valor_Contrato'].max(), 10)]
             )
        )

        fig_line_sector_cumulative.show()

        print("-" * 50) # Separator

        # 2. Non-Cumulative Total Contract Value by Sector Over Time (Monthly) - Divided into groups

        # Define sector groups based on visual inspection of previous plot (example grouping)
        # Adjust these groups based on actual visual patterns observed
        group1_sectors = ['transporte', 'deporte'] # Example: High value sectors
        group2_sectors = ['servicios públicos', 'salud', 'vivienda'] # Example: Medium value sectors
        group3_sectors = ['educación', 'medio ambiente', 'mejoramiento'] # Example: Lower value sectors
        group4_sectors = ['otro', 'seguridad', 'cultura', 'tecnología'] # Example: Other sectors


        sector_groups = {
            'Sectores de Alto Valor': group1_sectors,
            'Sectores de Valor Medio': group2_sectors,
            'Sectores de Valor Bajo': group3_sectors,
            'Otros Sectores': group4_sectors
        }

        for group_name, sectors in sector_groups.items():
             df_group = sector_value_by_month[sector_value_by_month['sector'].isin(sectors)].copy()

             if not df_group.empty:
                  fig_line_group = px.line(df_group, x='Date', y='Total_Valor_Contrato', color='sector',
                                            title=f'<b>Valor Total del Contrato a lo largo del Tiempo (Mensual) - {group_name} - OBRAS EN EL VALLE</b>',
                                            labels={'Date': 'Fecha', 'Total_Valor_Contrato': 'Valor Total del Contrato', 'sector': 'Sector'})

                  # Apply currency formatting to y-axis
                  fig_line_group.update_layout(
                       yaxis = dict(
                          tickmode = 'array',
                          # Generate ticks based on the data range of the current group
                          tickvals = [i for i in np.linspace(df_group['Total_Valor_Contrato'].min(), df_group['Total_Valor_Contrato'].max(), 10)], # Example: 10 ticks evenly spaced
                          ticktext = [format_cop_plotly(i) for i in np.linspace(df_group['Total_Valor_Contrato'].min(), df_group['Total_Valor_Contrato'].max(), 10)]
                       )
                  )
                  fig_line_group.show()
             else:
                 print(f"Skipping plot for group: {group_name}. No data for these sectors.")

    else:
        print(f"Skipping time series plots: No valid data points after cleaning for {CAMPO_VALOR} and {CAMPO_FI}.")
else:
    print("DataFrame df_raw is empty or required columns ('valor_contrato', 'fecha_inicio_ejecuci_n', 'sector') are not available.")

Removed 5 contract(s) with value exactly 3000000000.


--------------------------------------------------


In [ ]:
display(df_raw.info())

<class 'pandas.core.frame.DataFrame'>
Index: 21038 entries, 0 to 21042
Data columns (total 27 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   nivel_entidad                21038 non-null  object        
 1   codigo_entidad_en_secop      21038 non-null  object        
 2   nombre_de_la_entidad         21038 non-null  object        
 3   nit_de_la_entidad            21038 non-null  object        
 4   departamento_entidad         21038 non-null  object        
 5   municipio_entidad            21038 non-null  object        
 6   estado_del_proceso           21038 non-null  object        
 7   modalidad_de_contrataci_n    21038 non-null  object        
 8   objeto_a_contratar           21038 non-null  object        
 9   objeto_del_proceso           21038 non-null  object        
 10  tipo_de_contrato             21038 non-null  object        
 11  numero_del_contrato          21038 non-null  o

None

In [ ]:
import pandas as pd
import os

# Assuming df_raw is already defined

if not df_raw.empty:
    # Identify columns to drop by index (0-based)
    columns_to_drop_indices = [25, 26]

    # Get the names of the columns to drop
    all_columns = df_raw.columns.tolist()
    columns_to_drop_names = [all_columns[i] for i in columns_to_drop_indices if i < len(all_columns)]

    if columns_to_drop_names:
        print(f"Removing columns: {columns_to_drop_names}")
        # Drop the specified columns
        df_modified = df_raw.drop(columns=columns_to_drop_names, errors='ignore')

        # Define the output path for the new parquet file
        output_path_new = "/content/drive/MyDrive/secop_valle_obras_analisis_unibivariado.parquet"

        # Save the modified DataFrame to a new parquet file
        df_modified.to_parquet(output_path_new, index=False)

        print(f"DataFrame modified and saved to: {output_path_new}")

        # Update df_raw to the modified DataFrame for subsequent operations
        df_raw = df_modified
        print("\ndf_raw has been updated to the modified DataFrame.")
        display(df_raw.info())

    else:
        print("Specified column indices are out of bounds. No columns removed.")

else:
    print("DataFrame df_raw is empty. Skipping column removal and saving.")

#.-.-.-.-.    HASTA AQUÍ TODO FUNCIONA  .-.-.-.-.-.-.
#.-.-.-.-.    VIERNES                   .-.-.-.-..-.-
#.-.-.-.-.    29 DE AGOSTO DE 2025      .-.-.-.-.-.-.
#.-.-.-.-.    17:36 PM                  .-.-.-.-.-.-.
#.-.-.-.-.    muchos reportes y graficas       .-.-.-.-.-.-.

Removing columns: ['is_top_8_municipio', 'is_top_12_municipio']
DataFrame modified and saved to: /content/drive/MyDrive/secop_valle_obras_analisis_unibivariado.parquet

df_raw has been updated to the modified DataFrame.
<class 'pandas.core.frame.DataFrame'>
Index: 21038 entries, 0 to 21042
Data columns (total 25 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   nivel_entidad                21038 non-null  object        
 1   codigo_entidad_en_secop      21038 non-null  object        
 2   nombre_de_la_entidad         21038 non-null  object        
 3   nit_de_la_entidad            21038 non-null  object        
 4   departamento_entidad         21038 non-null  object        
 5   municipio_entidad            21038 non-null  object        
 6   estado_del_proceso           21038 non-null  object        
 7   modalidad_de_contrataci_n    21038 non-null  object        
 8   objeto_a_contratar   

None

In [ ]:
#INICIO DE ANALISIS DATASET DE FACTURAS

In [3]:
import requests
import pandas as pd
import json

# --- Configuración para el dataset de Facturas (Assuming these are defined in a previous cell) ---
# DATASET_ID_FACTURAS = "ibyt-yi2f"
# BASE_URL_FACTURAS   = f"https://www.datos.gov.co/resource/{DATASET_ID_FACTURAS}.json"
# HEADERS_FACTURAS    = {} # Add headers if needed, e.g., for APP_TOKEN

# Re-define if they might not be in the current environment
if 'BASE_URL_FACTURAS' not in globals():
    DATASET_ID_FACTURAS = "ibyt-yi2f"
    BASE_URL_FACTURAS   = f"https://www.datos.gov.co/resource/{DATASET_ID_FACTURAS}.json"
    HEADERS_FACTURAS    = {} # Or retrieve from os.getenv if applicable

print(f"Connecting to the API for dataset: {DATASET_ID_FACTURAS}")

# --- API Call to retrieve only id_contrato and valor_total ---
# Identify the correct field names for 'id_contrato' and 'valor_total'
# Assuming 'id_contrato' and 'valor_total' are the correct field names based on previous metadata check

id_contrato_field = 'id_contrato'
valor_total_field = 'valor_total'

print(f"Retrieving data with columns: {id_contrato_field}, {valor_total_field}")

# Use $select to specify the columns and $limit and $offset for pagination
params = {
    "$select": f"{id_contrato_field}, {valor_total_field}",
    "$limit": 50000,  # Adjust limit as needed, Socrata has limits
    "$offset": 0
}

all_data = []
limit = params['$limit']

while True:
    try:
        r = requests.get(BASE_URL_FACTURAS, params=params, headers=HEADERS_FACTURAS, timeout=600) # Increased timeout
        r.raise_for_status()
        data_page = r.json()
        if not data_page:
            break # No more data
        all_data.extend(data_page)
        if len(data_page) < limit:
            break # Last page
        params['$offset'] += limit
        print(f"Fetched {len(all_data)} records...") # Progress indicator

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data page at offset {params['$offset']}: {e}")
        break # Stop fetching on error

# Load the data into a pandas DataFrame
if all_data:
    df_facturas = pd.DataFrame(all_data)
    print(f"\nDataFrame df_facturas created with shape: {df_facturas.shape}")

    # Display the first few rows and info
    print("\nFirst 5 rows of df_facturas:")
    display(df_facturas.head())

    print("\nInfo of df_facturas:")
    display(df_facturas.info())

else:
    print("\nNo data retrieved from the API.")
    df_facturas = pd.DataFrame() # Create an empty DataFrame if no data is retrieved

Connecting to the API for dataset: ibyt-yi2f
Retrieving data with columns: id_contrato, valor_total
Fetched 50000 records...
Fetched 100000 records...
Fetched 150000 records...
Fetched 200000 records...
Fetched 250000 records...
Fetched 300000 records...
Fetched 350000 records...
Fetched 400000 records...
Fetched 450000 records...
Fetched 500000 records...
Fetched 550000 records...
Fetched 600000 records...
Fetched 650000 records...
Fetched 700000 records...
Fetched 750000 records...
Fetched 800000 records...
Fetched 850000 records...
Fetched 900000 records...
Fetched 950000 records...
Fetched 1000000 records...
Fetched 1050000 records...
Fetched 1100000 records...
Fetched 1150000 records...
Fetched 1200000 records...
Fetched 1250000 records...
Fetched 1300000 records...
Fetched 1350000 records...
Fetched 1400000 records...
Fetched 1450000 records...
Fetched 1500000 records...
Fetched 1550000 records...
Fetched 1600000 records...
Fetched 1650000 records...
Fetched 1700000 records...
Fe

,id_contrato,valor_total
0,CO1.PCCNTR.142707,100000000.000000
1,CO1.PCCNTR.142707,200576832.000000
2,CO1.PCCNTR.142707,65000000.000000
3,CO1.PCCNTR.122114,1412434.000000
4,CO1.PCCNTR.131613,3000000.000000



Info of df_facturas:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16879267 entries, 0 to 16879266
Data columns (total 2 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   id_contrato  object
 1   valor_total  object
dtypes: object(2)
memory usage: 257.6+ MB


None

In [4]:
import pandas as pd

# Check if df_facturas is loaded and not empty
if 'df_facturas' in globals() and not df_facturas.empty:
    # Ensure required columns exist and valor_total is numeric
    required_cols = ['id_contrato', 'valor_total']
    if all(col in df_facturas.columns for col in required_cols):
        # Convert 'valor_total' to numeric, coercing errors to NaN
        df_facturas['valor_total'] = pd.to_numeric(df_facturas['valor_total'], errors='coerce')

        # Drop rows where 'valor_total' is NaN after conversion
        df_facturas_cleaned = df_facturas.dropna(subset=['valor_total']).copy()

        print("Aggregating df_facturas by 'id_contrato'...")

        # Group by 'id_contrato' and aggregate
        # Sum 'valor_total' and count occurrences of 'id_contrato'
        df_facturas_agg = df_facturas_cleaned.groupby('id_contrato').agg(
            total_valor_total=('valor_total', 'sum'),
            cantidad_facturas=('id_contrato', 'size') # Using size to count occurrences
        ).reset_index()

        print(f"Aggregation complete. New DataFrame df_facturas_agg created with shape: {df_facturas_agg.shape}")

        # Display the first few rows of the aggregated DataFrame
        print("\nFirst 5 rows of the aggregated DataFrame:")
        display(df_facturas_agg.head())

        # Display info of the aggregated DataFrame
        print("\nInfo of the aggregated DataFrame:")
        display(df_facturas_agg.info())

        # Check if the number of rows in the aggregated DataFrame equals the number of unique id_contrato in the original (cleaned) data
        unique_id_contrato_count_cleaned = df_facturas_cleaned['id_contrato'].nunique()
        print(f"\nNumber of unique 'id_contrato' in the cleaned original data: {unique_id_contrato_count_cleaned}")
        print(f"Number of rows in the aggregated DataFrame: {len(df_facturas_agg)}")

    else:
        missing_cols = [col for col in required_cols if col not in df_facturas.columns]
        print(f"Required columns {missing_cols} not found in df_facturas. Skipping aggregation.")
else:
    print("DataFrame df_facturas is not loaded or is empty. Skipping aggregation.")

Aggregating df_facturas by 'id_contrato'...
Aggregation complete. New DataFrame df_facturas_agg created with shape: (2817589, 3)

First 5 rows of the aggregated DataFrame:


,id_contrato,total_valor_total,cantidad_facturas
0,CO1.PCCNTR.1000001,21013391.0,18
1,CO1.PCCNTR.100001,21386400.0,9
2,CO1.PCCNTR.100002,27745000.0,13
3,CO1.PCCNTR.100003,45283420.0,13
4,CO1.PCCNTR.1000504,24521210.0,7



Info of the aggregated DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2817589 entries, 0 to 2817588
Data columns (total 3 columns):
 #   Column             Dtype  
---  ------             -----  
 0   id_contrato        object 
 1   total_valor_total  float64
 2   cantidad_facturas  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 64.5+ MB


None


Number of unique 'id_contrato' in the cleaned original data: 2817589
Number of rows in the aggregated DataFrame: 2817589


In [11]:
import pandas as pd

# Check if both dataframes are loaded and not empty
if 'df_raw' in globals() and not df_raw.empty and 'df_facturas_agg' in globals() and not df_facturas_agg.empty:
    # Ensure the relevant columns exist in both dataframes
    if 'numero_del_contrato' in df_raw.columns and 'id_contrato' in df_facturas_agg.columns:
        print("Checking for common contract IDs between df_raw and df_facturas_agg...")

        # Get the unique contract IDs from both dataframes
        unique_contratos_raw = set(df_raw['numero_del_contrato'].unique())
        unique_contratos_facturas_agg = set(df_facturas_agg['id_contrato'].unique())

        # Find the intersection of the two sets
        common_contratos = unique_contratos_raw.intersection(unique_contratos_facturas_agg)

        # Count the number of common contract IDs
        num_common_contratos = len(common_contratos)

        print(f"\nNumber of contracts (numero_del_contrato in df_raw) also present in df_facturas_agg (id_contrato): {num_common_contratos}")

    else:
        missing_cols = []
        if 'numero_del_contrato' not in df_raw.columns:
            missing_cols.append('numero_del_contrato in df_raw')
        if 'id_contrato' not in df_facturas_agg.columns:
            missing_cols.append('id_contrato in df_facturas_agg')
        print(f"Required columns not found: {missing_cols}. Skipping the check.")

else:
    print("Either df_raw or df_facturas_agg is not loaded or is empty. Skipping the check.")

Checking for common contract IDs between df_raw and df_facturas_agg...

Number of contracts (numero_del_contrato in df_raw) also present in df_facturas_agg (id_contrato): 449
